In [1]:
import json
import requests
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import geopandas as gpd
import seaborn as sns
import numpy as np
pd.set_option('display.max_columns', None)

## Obtain and read the building permit data set for all the places in Midwest region in 2015

In [2]:
df = pd.read_table("https://www2.census.gov/econ/bps/Place/Midwest%20Region/mw2015a.txt", sep=",",
                  skipinitialspace=True) #read the online table

# the web data table has some format issue, this block of code is to address that issue
df["Unnamed 41"] = np.nan
df = df.shift(1, axis=1)
df.Survey = df.index
df.index = np.arange(len(df))

dict_names = {}
second  = df.iloc[0].fillna("")
for i in range(len(df.columns)):
    first = df.columns[i]
    if first.startswith("Unnamed"):
        if df.columns[i-1].startswith("Unnamed"):
            first = df.columns[i+1]
        else:
            first = df.columns[i-1]
    #         new = first+second[i]
    # else:
    new = first+" "+second[i]
    dict_names[df.columns[i]]= new
df = df.rename(columns = dict_names).drop([0])

# show the first five records
df.head()

,Survey Date,State Code,6-Digit ID,County Code,Census Place Code,FIPS Place Code,FIPS MCD Code,Pop,CSA Code,CBSA Code,Footnote Code,Central City,Zip Code,Region Code,Division Code,Number of Months Rep,Place Name,Place Bldgs,1-unit Units,1-unit Value,2-units Bldgs,2-units Units,2-units Value,3-4 units Bldgs,3-4 units Units,3-4 units Value,5+ units Bldgs,5+ units Units,5+ units Value,1-unit rep Bldgs,1-unit rep Units,1-unit rep Value,2-units rep Bldgs,2-units rep Units,2-units rep Value,3-4 units rep Bldgs,3-4 units rep Units,3-4 units rep Value,5+ units rep Bldgs,5+ units rep Units,5+ units rep Value
1,2015,17,001000,095,0005,00113,11891,3319.0,999,99999,NaN,NaN,61410,2,3,12,Abingdon,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2015,17,002800,043,0015,00243,00250,36942.0,176,16980,NaN,NaN,601012786,2,3,11,Addison village,4,4,1225000,0,0,0,0,0,0,0,0,0,4,4,1225000,0,0,0,0,0,0,0,0,0
3,2015,17,004900,195,0025,00516,00529,891.0,999,99999,NaN,NaN,61230,2,3,12,Albany village,1,1,150000,0,0,0,0,0,0,0,0,0,1,1,150000,0,0,0,0,0,0,0,0,0
4,2015,17,005200,027,0030,00555,44667,1190.0,476,41180,NaN,NaN,62215,2,3,12,Albers village,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,2015,17,006100,131,0040,00646,48398,3640.0,209,19340,NaN,NaN,61231,2,3,12,Aledo,1,1,250000,0,0,0,0,0,0,0,0,0,1,1,250000,0,0,0,0,0,0,0,0,0


### Two types of housing units:

* Single-family units: 1-unit rep Units
* Multifamily units: > 1 (need to add those fields up)
    * 2-units rep Units
    * 3-4 units rep Units
    * 5+ units rep Units

**Obtain and read the building permit data set for all the places in Midwest region in 2016**

In [3]:
df1 = pd.read_table("https://www2.census.gov/econ/bps/Place/Midwest%20Region/mw2016a.txt", sep=",",
                  skipinitialspace=True) #read the online table

# the web data table has some format issue, this block of code is to address that issue
df1["Unnamed 41"] = np.nan
df1 = df1.shift(1, axis=1)
df1.Survey = df1.index
df1.index = np.arange(len(df1))

dict_names = {}
second  = df1.iloc[0].fillna("")
for i in range(len(df1.columns)):
    first = df1.columns[i]
    if first.startswith("Unnamed"):
        if df1.columns[i-1].startswith("Unnamed"):
            first = df1.columns[i+1]
        else:
            first = df1.columns[i-1]
    #         new = first+second[i]
    # else:
    new = first+" "+second[i]
    dict_names[df1.columns[i]]= new
df1 = df1.rename(columns = dict_names).drop([0])

# show the first five records
df1.head()

,Survey Date,State Code,6-Digit ID,County Code,Census Place Code,FIPS Place Code,FIPS MCD Code,Pop,CSA Code,CBSA Code,Footnote Code,Central City,Zip Code,Region Code,Division Code,Number of Months Rep,Place Name,Place Bldgs,1-unit Units,1-unit Value,2-units Bldgs,2-units Units,2-units Value,3-4 units Bldgs,3-4 units Units,3-4 units Value,5+ units Bldgs,5+ units Units,5+ units Value,1-unit rep Bldgs,1-unit rep Units,1-unit rep Value,2-units rep Bldgs,2-units rep Units,2-units rep Value,3-4 units rep Bldgs,3-4 units rep Units,3-4 units rep Value,5+ units rep Bldgs,5+ units rep Units,5+ units rep Value
1,2016,17,001000,095,0005,00113,11891,3319.0,999,99999,NaN,NaN,61410,2,3,12,Abingdon,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2016,17,002800,043,0015,00243,00250,36942.0,176,16980,NaN,NaN,601012786,2,3,12,Addison village,3,3,950000,0,0,0,0,0,0,0,0,0,3,3,950000,0,0,0,0,0,0,0,0,0
3,2016,17,004900,195,0025,00516,00529,891.0,999,99999,NaN,NaN,61230,2,3,12,Albany village,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,2016,17,005200,027,0030,00555,44667,1190.0,476,41180,NaN,NaN,62215,2,3,12,Albers village,1,1,60000,0,0,0,0,0,0,0,0,0,1,1,60000,0,0,0,0,0,0,0,0,0
5,2016,17,006100,131,0040,00646,48398,3640.0,209,19340,NaN,NaN,61231,2,3,12,Aledo,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


**Obtain and read the building permit data set for all the places in Midwest region in 2017**

In [4]:
df2 = pd.read_table("https://www2.census.gov/econ/bps/Place/Midwest%20Region/mw2017a.txt", sep=",",
                  skipinitialspace=True) #read the online table

# the web data table has some format issue, this block of code is to address that issue
df2["Unnamed 41"] = np.nan
df2 = df2.shift(1, axis=1)
df2.Survey = df2.index
df2.index = np.arange(len(df2))

dict_names = {}
second  = df2.iloc[0].fillna("")
for i in range(len(df2.columns)):
    first = df2.columns[i]
    if first.startswith("Unnamed"):
        if df2.columns[i-1].startswith("Unnamed"):
            first = df2.columns[i+1]
        else:
            first = df2.columns[i-1]
    #         new = first+second[i]
    # else:
    new = first+" "+second[i]
    dict_names[df2.columns[i]]= new
df2 = df2.rename(columns = dict_names).drop([0])

# show the first five records
df2.head()

,Survey Date,State Code,6-Digit ID,County Code,Census Place Code,FIPS Place Code,FIPS MCD Code,Pop,CSA Code,CBSA Code,Footnote Code,Central City,Zip Code,Region Code,Division Code,Number of Months Rep,Place Name,Place Bldgs,1-unit Units,1-unit Value,2-units Bldgs,2-units Units,2-units Value,3-4 units Bldgs,3-4 units Units,3-4 units Value,5+ units Bldgs,5+ units Units,5+ units Value,1-unit rep Bldgs,1-unit rep Units,1-unit rep Value,2-units rep Bldgs,2-units rep Units,2-units rep Value,3-4 units rep Bldgs,3-4 units rep Units,3-4 units rep Value,5+ units rep Bldgs,5+ units rep Units,5+ units rep Value
1,2017,17,001000,095,0005,00113,11891,3319.0,999,99999,NaN,NaN,61410,2,3,12,Abingdon,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2017,17,002800,043,0015,00243,00250,36942.0,176,16980,NaN,NaN,601012786,2,3,12,Addison village,16,16,3850000,0,0,0,0,0,0,0,0,0,16,16,3850000,0,0,0,0,0,0,0,0,0
3,2017,17,004900,195,0025,00516,00529,891.0,999,99999,NaN,NaN,61230,2,3,12,Albany village,2,2,560000,0,0,0,0,0,0,0,0,0,2,2,560000,0,0,0,0,0,0,0,0,0
4,2017,17,005200,027,0030,00555,44667,1190.0,476,41180,NaN,NaN,62215,2,3,12,Albers village,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,2017,17,006100,131,0040,00646,48398,3640.0,209,19340,NaN,NaN,61231,2,3,12,Aledo,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


**Obtain and read the building permit data set for all the places in Midwest region in 2018**

In [5]:
df3 = pd.read_table("https://www2.census.gov/econ/bps/Place/Midwest%20Region/mw2018a.txt", sep=",",
                  skipinitialspace=True) #read the online table

# the web data table has some format issue, this block of code is to address that issue
df3["Unnamed 41"] = np.nan
df3 = df3.shift(1, axis=1)
df3.Survey = df3.index
df3.index = np.arange(len(df3))

dict_names = {}
second  = df3.iloc[0].fillna("")
for i in range(len(df3.columns)):
    first = df3.columns[i]
    if first.startswith("Unnamed"):
        if df3.columns[i-1].startswith("Unnamed"):
            first = df3.columns[i+1]
        else:
            first = df3.columns[i-1]
    #         new = first+second[i]
    # else:
    new = first+" "+second[i]
    dict_names[df3.columns[i]]= new
df3 = df3.rename(columns = dict_names).drop([0])

# show the first five records
df3.head()

,Survey Date,State Code,6-Digit ID,County Code,Census Place Code,FIPS Place Code,FIPS MCD Code,Pop,CSA Code,CBSA Code,Footnote Code,Central City,Zip Code,Region Code,Division Code,Number of Months Rep,Place Name,Place Bldgs,1-unit Units,1-unit Value,2-units Bldgs,2-units Units,2-units Value,3-4 units Bldgs,3-4 units Units,3-4 units Value,5+ units Bldgs,5+ units Units,5+ units Value,1-unit rep Bldgs,1-unit rep Units,1-unit rep Value,2-units rep Bldgs,2-units rep Units,2-units rep Value,3-4 units rep Bldgs,3-4 units rep Units,3-4 units rep Value,5+ units rep Bldgs,5+ units rep Units,5+ units rep Value
1,2018,17,001000,095,0005,00113,11891,3319.0,999,99999,NaN,NaN,61410,2,3,12,Abingdon,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2018,17,002800,043,0015,00243,00250,36942.0,176,16980,NaN,NaN,601012786,2,3,12,Addison village,30,30,7175000,0,0,0,0,0,0,0,0,0,30,30,7175000,0,0,0,0,0,0,0,0,0
3,2018,17,004900,195,0025,00516,00529,891.0,999,99999,NaN,NaN,61230,2,3,12,Albany village,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,2018,17,005200,027,0030,00555,44667,1190.0,476,41180,NaN,NaN,62215,2,3,12,Albers village,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,2018,17,006100,131,0040,00646,48398,3640.0,209,19340,NaN,NaN,61231,2,3,12,Aledo,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


**Obtain and read the building permit data set for all the places in Midwest region in 2019**

In [6]:
df4 = pd.read_table("https://www2.census.gov/econ/bps/Place/Midwest%20Region/mw2019a.txt", sep=",",
                  skipinitialspace=True) #read the online table

# the web data table has some format issue, this block of code is to address that issue
df4["Unnamed 41"] = np.nan
df4 = df4.shift(1, axis=1)
df4.Survey = df4.index
df4.index = np.arange(len(df4))

dict_names = {}
second  = df4.iloc[0].fillna("")
for i in range(len(df4.columns)):
    first = df4.columns[i]
    if first.startswith("Unnamed"):
        if df4.columns[i-1].startswith("Unnamed"):
            first = df4.columns[i+1]
        else:
            first = df4.columns[i-1]
    #         new = first+second[i]
    # else:
    new = first+" "+second[i]
    dict_names[df4.columns[i]]= new
df4 = df4.rename(columns = dict_names).drop([0])

# show the first five records
df4.head()

,Survey Date,State Code,6-Digit ID,County Code,Census Place Code,FIPS Place Code,FIPS MCD Code,Pop,CSA Code,CBSA Code,Footnote Code,Central City,Zip Code,Region Code,Division Code,Number of Months Rep,Place Name,Place Bldgs,1-unit Units,1-unit Value,2-units Bldgs,2-units Units,2-units Value,3-4 units Bldgs,3-4 units Units,3-4 units Value,5+ units Bldgs,5+ units Units,5+ units Value,1-unit rep Bldgs,1-unit rep Units,1-unit rep Value,2-units rep Bldgs,2-units rep Units,2-units rep Value,3-4 units rep Bldgs,3-4 units rep Units,3-4 units rep Value,5+ units rep Bldgs,5+ units rep Units,5+ units rep Value
1,2019,17,001000,095,0005,00113,11891,3319.0,999,99999,NaN,NaN,61410,2,3,12,Abingdon,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2019,17,002800,043,0015,00243,00250,36942.0,176,16980,NaN,NaN,601012786,2,3,12,Addison village,28,28,6350625,0,0,0,0,0,0,0,0,0,28,28,6350625,0,0,0,0,0,0,0,0,0
3,2019,17,004900,195,0025,00516,00529,891.0,999,99999,NaN,NaN,61230,2,3,12,Albany village,1,1,427000,0,0,0,0,0,0,0,0,0,1,1,427000,0,0,0,0,0,0,0,0,0
4,2019,17,005200,027,0030,00555,44667,1190.0,476,41180,NaN,NaN,62215,2,3,12,Albers village,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,2019,17,006100,131,0040,00646,48398,3640.0,209,19340,NaN,NaN,61231,2,3,12,Aledo,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [7]:
#combining all years data 2015-2019 for midwest region
vertical_concat = pd.concat([df,df1, df2,df3,df4], axis=0)

In [8]:
#concatenated dfs for midwest region for 2015-2019
dfmw=vertical_concat

In [9]:
dfmw=dfmw.reset_index()

In [10]:
# Drop a specific column, e.g., 'Place Name'
dfmw = dfmw.drop(columns=['index'])

In [11]:
dfmw

,Survey Date,State Code,6-Digit ID,County Code,Census Place Code,FIPS Place Code,FIPS MCD Code,Pop,CSA Code,CBSA Code,Footnote Code,Central City,Zip Code,Region Code,Division Code,Number of Months Rep,Place Name,Place Bldgs,1-unit Units,1-unit Value,2-units Bldgs,2-units Units,2-units Value,3-4 units Bldgs,3-4 units Units,3-4 units Value,5+ units Bldgs,5+ units Units,5+ units Value,1-unit rep Bldgs,1-unit rep Units,1-unit rep Value,2-units rep Bldgs,2-units rep Units,2-units rep Value,3-4 units rep Bldgs,3-4 units rep Units,3-4 units rep Value,5+ units rep Bldgs,5+ units rep Units,5+ units rep Value
0,2015,17,001000,095,0005,00113,11891,3319.0,999,99999,NaN,NaN,61410,2,3,12,Abingdon,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2015,17,002800,043,0015,00243,00250,36942.0,176,16980,NaN,NaN,601012786,2,3,11,Addison village,4,4,1225000,0,0,0,0,0,0,0,0,0,4,4,1225000,0,0,0,0,0,0,0,0,0
2,2015,17,004900,195,0025,00516,00529,891.0,999,99999,NaN,NaN,61230,2,3,12,Albany village,1,1,150000,0,0,0,0,0,0,0,0,0,1,1,150000,0,0,0,0,0,0,0,0,0
3,2015,17,005200,027,0030,00555,44667,1190.0,476,41180,NaN,NaN,62215,2,3,12,Albers village,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,2015,17,006100,131,0040,00646,48398,3640.0,209,19340,NaN,NaN,61231,2,3,12,Aledo,1,1,250000,0,0,0,0,0,0,0,0,0,1,1,250000,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40022,2019,55,979000,043,NaN,00000,89250,346.0,999,99999,NaN,NaN,53801,2,3,12,Wyalusing town,4,4,949000,0,0,0,0,0,0,0,0,0,4,4,949000,0,0,0,0,0,0,0,0,0
40023,2019,55,979500,081,2880,00000,89275,147.0,999,99999,NaN,NaN,54671,2,3,12,Wyeville village,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
40024,2019,55,980500,021,2885,00000,89300,768.0,357,31540,NaN,NaN,53969,2,3,12,Wyocena village,1,1,178000,0,0,0,0,0,0,0,0,0,1,1,178000,0,0,0,0,0,0,0,0,0
40025,2019,55,982000,019,NaN,00000,89425,886.0,999,99999,NaN,NaN,54446,2,3,0,York town,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [12]:
import pandas as pd

# Assuming your data is in a CSV file named 'data.csv'
data = dfmw

# Select the columns of interest
selected_columns = ['FIPS Place Code','Zip Code','Survey Date', '1-unit rep Units', 'Place Name']

# Extract the desired data
extracted_datas = data[selected_columns]



**filtering  the column with 1-unit rep unit data**

In [13]:
#choosing the column with 1-unit rep unit data
extracted_datas

,FIPS Place Code,Zip Code,Survey Date,1-unit rep Units,Place Name
0,00113,61410,2015,0,Abingdon
1,00243,601012786,2015,4,Addison village
2,00516,61230,2015,1,Albany village
3,00555,62215,2015,0,Albers village
4,00646,61231,2015,1,Aledo
...,...,...,...,...,...
40022,00000,53801,2019,4,Wyalusing town
40023,00000,54671,2019,0,Wyeville village
40024,00000,53969,2019,1,Wyocena village
40025,00000,54446,2019,0,York town


In [14]:
# Group by 'Zip Code', 'FIPS Place Code', and 'Place Name', and sum '1-unit rep Units'
grouped_df = extracted_datas.groupby(['Zip Code', 'FIPS Place Code', 'Place Name', 'Survey Date'])['1-unit rep Units'].sum().reset_index()

# Pivot the DataFrame to have years as columns
pivot_df = grouped_df.pivot_table(index=['Zip Code', 'FIPS Place Code', 'Place Name'], columns='Survey Date', values='1-unit rep Units', fill_value=0)

# Reset the index to have 'Zip Code', 'FIPS Place Code', and 'Place Name' as columns
pivot_df.reset_index(inplace=True)

# Rename the columns
pivot_df.columns = ['Zip Code', 'FIPS Place Code', 'Place Name', '2015', '2016', '2017', '2018', '2019']


# Display the pivot DataFrame
print(pivot_df)

         Zip Code FIPS Place Code                           Place Name  2015  \
0     15454                48855                               Manilla     0   
1     19372                00000                       Tiffin township     0   
2     42112                00000                       Milton township    14   
3     43003                00000                    Westfield township     0   
4     43011                00000                      Hilliar township     9   
...           ...             ...                                  ...   ...   
8494  69358                32830                       Morrill village     0   
8495  69360                42775                             Rushville     0   
8496  693600039            99990   Sheridan County Unincorporated Area     0   
8497  69361                44245                           Scottsbluff     9   
8498  69367                52925                       Whitney village     0   

      2016  2017  2018  2019  
0       

# **Building permits for midwest region fips place code for single units from 2015-2019**

In [15]:
pivot_df

,Zip Code,FIPS Place Code,Place Name,2015,2016,2017,2018,2019
0,15454,48855,Manilla,0,0,0,1,0
1,19372,00000,Tiffin township,0,0,0,3,0
2,42112,00000,Milton township,14,11,15,11,12
3,43003,00000,Westfield township,0,0,0,0,6
4,43011,00000,Hilliar township,9,10,11,10,14
...,...,...,...,...,...,...,...,...
8494,69358,32830,Morrill village,0,1,0,1,0
8495,69360,42775,Rushville,0,0,0,0,0
8496,693600039,99990,Sheridan County Unincorporated Area,0,0,0,0,0
8497,69361,44245,Scottsbluff,9,10,10,8,5


In [16]:
dfmw

,Survey Date,State Code,6-Digit ID,County Code,Census Place Code,FIPS Place Code,FIPS MCD Code,Pop,CSA Code,CBSA Code,Footnote Code,Central City,Zip Code,Region Code,Division Code,Number of Months Rep,Place Name,Place Bldgs,1-unit Units,1-unit Value,2-units Bldgs,2-units Units,2-units Value,3-4 units Bldgs,3-4 units Units,3-4 units Value,5+ units Bldgs,5+ units Units,5+ units Value,1-unit rep Bldgs,1-unit rep Units,1-unit rep Value,2-units rep Bldgs,2-units rep Units,2-units rep Value,3-4 units rep Bldgs,3-4 units rep Units,3-4 units rep Value,5+ units rep Bldgs,5+ units rep Units,5+ units rep Value
0,2015,17,001000,095,0005,00113,11891,3319.0,999,99999,NaN,NaN,61410,2,3,12,Abingdon,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2015,17,002800,043,0015,00243,00250,36942.0,176,16980,NaN,NaN,601012786,2,3,11,Addison village,4,4,1225000,0,0,0,0,0,0,0,0,0,4,4,1225000,0,0,0,0,0,0,0,0,0
2,2015,17,004900,195,0025,00516,00529,891.0,999,99999,NaN,NaN,61230,2,3,12,Albany village,1,1,150000,0,0,0,0,0,0,0,0,0,1,1,150000,0,0,0,0,0,0,0,0,0
3,2015,17,005200,027,0030,00555,44667,1190.0,476,41180,NaN,NaN,62215,2,3,12,Albers village,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,2015,17,006100,131,0040,00646,48398,3640.0,209,19340,NaN,NaN,61231,2,3,12,Aledo,1,1,250000,0,0,0,0,0,0,0,0,0,1,1,250000,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40022,2019,55,979000,043,NaN,00000,89250,346.0,999,99999,NaN,NaN,53801,2,3,12,Wyalusing town,4,4,949000,0,0,0,0,0,0,0,0,0,4,4,949000,0,0,0,0,0,0,0,0,0
40023,2019,55,979500,081,2880,00000,89275,147.0,999,99999,NaN,NaN,54671,2,3,12,Wyeville village,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
40024,2019,55,980500,021,2885,00000,89300,768.0,357,31540,NaN,NaN,53969,2,3,12,Wyocena village,1,1,178000,0,0,0,0,0,0,0,0,0,1,1,178000,0,0,0,0,0,0,0,0,0
40025,2019,55,982000,019,NaN,00000,89425,886.0,999,99999,NaN,NaN,54446,2,3,0,York town,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


# Filtering for multiple units for midwest region

In [17]:
import pandas as pd


data = dfmw

# Select the columns of interest
selected_columnsm = ['FIPS Place Code','Zip Code','Survey Date', '2-units rep Units','3-4 units rep Units','5+ units rep Units', 'Place Name']

# Extract the desired data
extracted_datam = data[selected_columnsm]

In [18]:
extracted_datam

,FIPS Place Code,Zip Code,Survey Date,2-units rep Units,3-4 units rep Units,5+ units rep Units,Place Name
0,00113,61410,2015,0,0,0,Abingdon
1,00243,601012786,2015,0,0,0,Addison village
2,00516,61230,2015,0,0,0,Albany village
3,00555,62215,2015,0,0,0,Albers village
4,00646,61231,2015,0,0,0,Aledo
...,...,...,...,...,...,...,...
40022,00000,53801,2019,0,0,0,Wyalusing town
40023,00000,54671,2019,0,0,0,Wyeville village
40024,00000,53969,2019,0,0,0,Wyocena village
40025,00000,54446,2019,0,0,0,York town


In [19]:
# Group by 'Zip Code,' 'FIPS Place Code,' 'Place Name,' and 'Survey Date,' and sum the units columns
grouped_dfm = extracted_datam.groupby(['Zip Code', 'FIPS Place Code', 'Place Name', 'Survey Date']).agg({
    '2-units rep Units': 'sum',
    '3-4 units rep Units': 'sum',
    '5+ units rep Units': 'sum'
}).reset_index()

# Create a new column 'multi unit' by summing the '2-units rep Units,' '3-4 units rep Units,' and '5+ units rep Units'
grouped_dfm['multi unit'] = grouped_dfm['2-units rep Units'] + grouped_dfm['3-4 units rep Units'] + grouped_dfm['5+ units rep Units']


# Display the grouped DataFrame
print(grouped_dfm)





          Zip Code FIPS Place Code       Place Name Survey Date  \
0      15454                48855           Manilla        2015   
1      15454                48855           Manilla        2016   
2      15454                48855           Manilla        2017   
3      15454                48855           Manilla        2018   
4      15454                48855           Manilla        2019   
...            ...             ...              ...         ...   
40004  69367                52925   Whitney village        2015   
40005  69367                52925   Whitney village        2016   
40006  69367                52925   Whitney village        2017   
40007  69367                52925   Whitney village        2018   
40008  69367                52925   Whitney village        2019   

      2-units rep Units 3-4 units rep Units 5+ units rep Units multi unit  
0                     0                   0                  0        000  
1                     0                   0

In [20]:
grouped_dfm

,Zip Code,FIPS Place Code,Place Name,Survey Date,2-units rep Units,3-4 units rep Units,5+ units rep Units,multi unit
0,15454,48855,Manilla,2015,0,0,0,000
1,15454,48855,Manilla,2016,0,0,0,000
2,15454,48855,Manilla,2017,0,0,0,000
3,15454,48855,Manilla,2018,0,0,0,000
4,15454,48855,Manilla,2019,0,0,0,000
...,...,...,...,...,...,...,...,...
40004,69367,52925,Whitney village,2015,0,0,0,000
40005,69367,52925,Whitney village,2016,0,0,0,000
40006,69367,52925,Whitney village,2017,0,0,0,000
40007,69367,52925,Whitney village,2018,0,0,0,000


In [21]:
# Pivot the DataFrame to have years as columns
pivot_dfm = grouped_dfm.pivot_table(index=['Zip Code', 'FIPS Place Code', 'Place Name'], columns='Survey Date', values='multi unit', fill_value=0)

# Reset the index to have 'Zip Code', 'FIPS Place Code', and 'Place Name' as columns
pivot_dfm.reset_index(inplace=True)

# Rename the columns
pivot_dfm.columns = ['Zip Code', 'FIPS Place Code', 'Place Name', '2015', '2016', '2017', '2018', '2019']

# Save the pivot DataFrame to a CSV file
pivot_dfm.to_csv('building_permits_grouped_2015-2019.csv', index=False)

# Display the pivot DataFrame
print(pivot_dfm)

         Zip Code FIPS Place Code                           Place Name  2015  \
0     15454                48855                               Manilla     0   
1     19372                00000                       Tiffin township     0   
2     42112                00000                       Milton township     0   
3     43003                00000                    Westfield township     0   
4     43011                00000                      Hilliar township     0   
...           ...             ...                                  ...   ...   
8494  69358                32830                       Morrill village     0   
8495  69360                42775                             Rushville     0   
8496  693600039            99990   Sheridan County Unincorporated Area     0   
8497  69361                44245                           Scottsbluff     0   
8498  69367                52925                       Whitney village     0   

      2016  2017  2018  2019  
0       

# **Building permits for fips place code for multi units for midwest region from 2015-2019**

In [22]:
pivot_dfm

,Zip Code,FIPS Place Code,Place Name,2015,2016,2017,2018,2019
0,15454,48855,Manilla,0,0,0,0,0
1,19372,00000,Tiffin township,0,0,0,0,0
2,42112,00000,Milton township,0,0,0,0,0
3,43003,00000,Westfield township,0,0,0,0,0
4,43011,00000,Hilliar township,0,0,0,0,0
...,...,...,...,...,...,...,...,...
8494,69358,32830,Morrill village,0,0,0,0,0
8495,69360,42775,Rushville,0,0,0,0,0
8496,693600039,99990,Sheridan County Unincorporated Area,0,0,0,0,0
8497,69361,44245,Scottsbluff,0,12,200,0,0


# Obtain and read the building permit data set for all the places in Northeast region

In [23]:
#Obtain and read the building permit data set for all the places in Northeast Region

In [24]:
#https://www2.census.gov/econ/bps/Place/Northeast%20Region/ne2015a.txt

In [25]:
dfn1 = pd.read_table("https://www2.census.gov/econ/bps/Place/Northeast%20Region/ne2015a.txt", sep=",",
                  skipinitialspace=True) #read the online table

# the web data table has some format issue, this block of code is to address that issue
dfn1["Unnamed 41"] = np.nan
dfn1 = dfn1.shift(1, axis=1)
dfn1.Survey = dfn1.index
dfn1.index = np.arange(len(dfn1))

dict_names = {}
second  = dfn1.iloc[0].fillna("")
for i in range(len(dfn1.columns)):
    first = dfn1.columns[i]
    if first.startswith("Unnamed"):
        if dfn1.columns[i-1].startswith("Unnamed"):
            first = dfn1.columns[i+1]
        else:
            first = dfn1.columns[i-1]
    #         new = first+second[i]
    # else:
    new = first+" "+second[i]
    dict_names[dfn1.columns[i]]= new
dfn1 = dfn1.rename(columns = dict_names).drop([0])

# show the first five records
dfn1.head()

,Survey Date,State Code,6-Digit ID,County Code,Census Place Code,FIPS Place Code,FIPS MCD Code,Pop,CSA Code,CBSA Code,Footnote Code,Central City,Zip Code,Region Code,Division Code,Number of Months Rep,Place Name,Place Bldgs,1-unit Units,1-unit Value,2-units Bldgs,2-units Units,2-units Value,3-4 units Bldgs,3-4 units Units,3-4 units Value,5+ units Bldgs,5+ units Units,5+ units Value,1-unit rep Bldgs,1-unit rep Units,1-unit rep Value,2-units rep Bldgs,2-units rep Units,2-units rep Value,3-4 units rep Bldgs,3-4 units rep Units,3-4 units rep Value,5+ units rep Bldgs,5+ units rep Units,5+ units rep Value
1,2015,09,001000,013,NaN,00000,01080,3303.0,278,25540,NaN,NaN,06232,1,1,12,Andover town,4,4,754900,0,0,0,0,0,0,0,0,0,4,4,754900,0,0,0,0,0,0,0,0,0
2,2015,09,005000,009,0010,01150,01220,19249.0,408,35300,2,NaN,064011855,1,1,0,Ansonia,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,2015,09,009000,015,NaN,00000,01430,4317.0,148,49340,NaN,NaN,06278,1,1,12,Ashford town,2,2,281500,0,0,0,0,0,0,0,0,0,2,2,281500,0,0,0,0,0,0,0,0,0
4,2015,09,013000,003,NaN,00000,02060,18098.0,278,25540,NaN,NaN,06001,1,1,12,Avon town,31,31,10995334,0,0,0,0,0,0,0,0,0,31,31,10995334,0,0,0,0,0,0,0,0,0
5,2015,09,021000,005,NaN,00000,02760,3799.0,999,99999,NaN,NaN,060633340,1,1,12,Barkhamsted town,3,3,1035150,0,0,0,0,0,0,0,0,0,3,3,1035150,0,0,0,0,0,0,0,0,0


In [26]:
dfn2 = pd.read_table("https://www2.census.gov/econ/bps/Place/Northeast%20Region/ne2016a.txt", sep=",",
                  skipinitialspace=True) #read the online table

# the web data table has some format issue, this block of code is to address that issue
dfn2["Unnamed 41"] = np.nan
dfn2 = dfn2.shift(1, axis=1)
dfn2.Survey = dfn2.index
dfn2.index = np.arange(len(dfn2))

dict_names = {}
second  = dfn2.iloc[0].fillna("")
for i in range(len(dfn2.columns)):
    first = dfn2.columns[i]
    if first.startswith("Unnamed"):
        if dfn2.columns[i-1].startswith("Unnamed"):
            first = dfn2.columns[i+1]
        else:
            first = dfn2.columns[i-1]
    #         new = first+second[i]
    # else:
    new = first+" "+second[i]
    dict_names[dfn2.columns[i]]= new
dfn2 = dfn2.rename(columns = dict_names).drop([0])

# show the first five records
dfn2.head()

,Survey Date,State Code,6-Digit ID,County Code,Census Place Code,FIPS Place Code,FIPS MCD Code,Pop,CSA Code,CBSA Code,Footnote Code,Central City,Zip Code,Region Code,Division Code,Number of Months Rep,Place Name,Place Bldgs,1-unit Units,1-unit Value,2-units Bldgs,2-units Units,2-units Value,3-4 units Bldgs,3-4 units Units,3-4 units Value,5+ units Bldgs,5+ units Units,5+ units Value,1-unit rep Bldgs,1-unit rep Units,1-unit rep Value,2-units rep Bldgs,2-units rep Units,2-units rep Value,3-4 units rep Bldgs,3-4 units rep Units,3-4 units rep Value,5+ units rep Bldgs,5+ units rep Units,5+ units rep Value
1,2016,09,001000,013,NaN,00000,01080,3303.0,278,25540,NaN,NaN,06232,1,1,12,Andover town,3,3,945000,0,0,0,0,0,0,0,0,0,3,3,945000,0,0,0,0,0,0,0,0,0
2,2016,09,005000,009,0010,01150,01220,19249.0,408,35300,2,NaN,064011855,1,1,0,Ansonia,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,2016,09,009000,015,NaN,00000,01430,4317.0,148,49340,NaN,NaN,06278,1,1,12,Ashford town,5,5,1004883,1,2,145000,0,0,0,0,0,0,5,5,1004883,1,2,145000,0,0,0,0,0,0
4,2016,09,013000,003,NaN,00000,02060,18098.0,278,25540,NaN,NaN,06001,1,1,12,Avon town,19,19,6756493,0,0,0,0,0,0,0,0,0,19,19,6756493,0,0,0,0,0,0,0,0,0
5,2016,09,021000,005,NaN,00000,02760,3799.0,999,99999,NaN,NaN,060633340,1,1,12,Barkhamsted town,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [27]:
dfn3 = pd.read_table("https://www2.census.gov/econ/bps/Place/Northeast%20Region/ne2017a.txt", sep=",",
                  skipinitialspace=True) #read the online table

# the web data table has some format issue, this block of code is to address that issue
dfn3["Unnamed 41"] = np.nan
dfn3 = dfn3.shift(1, axis=1)
dfn3.Survey = dfn3.index
dfn3.index = np.arange(len(dfn3))

dict_names = {}
second  = dfn3.iloc[0].fillna("")
for i in range(len(dfn3.columns)):
    first = dfn3.columns[i]
    if first.startswith("Unnamed"):
        if dfn3.columns[i-1].startswith("Unnamed"):
            first = dfn3.columns[i+1]
        else:
            first = dfn3.columns[i-1]
    #         new = first+second[i]
    # else:
    new = first+" "+second[i]
    dict_names[dfn3.columns[i]]= new
dfn3 = dfn3.rename(columns = dict_names).drop([0])

# show the first five records
dfn3.head()

,Survey Date,State Code,6-Digit ID,County Code,Census Place Code,FIPS Place Code,FIPS MCD Code,Pop,CSA Code,CBSA Code,Footnote Code,Central City,Zip Code,Region Code,Division Code,Number of Months Rep,Place Name,Place Bldgs,1-unit Units,1-unit Value,2-units Bldgs,2-units Units,2-units Value,3-4 units Bldgs,3-4 units Units,3-4 units Value,5+ units Bldgs,5+ units Units,5+ units Value,1-unit rep Bldgs,1-unit rep Units,1-unit rep Value,2-units rep Bldgs,2-units rep Units,2-units rep Value,3-4 units rep Bldgs,3-4 units rep Units,3-4 units rep Value,5+ units rep Bldgs,5+ units rep Units,5+ units rep Value
1,2017,09,001000,013,NaN,00000,01080,3303.0,278,25540,NaN,NaN,06232,1,1,12,Andover town,3,3,835000,0,0,0,0,0,0,0,0,0,3,3,835000,0,0,0,0,0,0,0,0,0
2,2017,09,005000,009,0010,01150,01220,19249.0,408,35300,2,NaN,064011855,1,1,12,Ansonia,0,0,0,3,6,697000,0,0,0,0,0,0,0,0,0,3,6,697000,0,0,0,0,0,0
3,2017,09,009000,015,NaN,00000,01430,4317.0,148,49340,NaN,NaN,06278,1,1,12,Ashford town,8,8,1395000,0,0,0,0,0,0,0,0,0,8,8,1395000,0,0,0,0,0,0,0,0,0
4,2017,09,013000,003,NaN,00000,02060,18098.0,278,25540,NaN,NaN,06001,1,1,7,Avon town,20,20,6555963,0,0,0,0,0,0,0,0,0,10,10,2999913,0,0,0,0,0,0,0,0,0
5,2017,09,021000,005,NaN,00000,02760,3799.0,999,99999,NaN,NaN,060633340,1,1,12,Barkhamsted town,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [28]:
dfn4 = pd.read_table("https://www2.census.gov/econ/bps/Place/Northeast%20Region/ne2018a.txt", sep=",",
                  skipinitialspace=True) #read the online table

# the web data table has some format issue, this block of code is to address that issue
dfn4["Unnamed 41"] = np.nan
dfn4 = dfn4.shift(1, axis=1)
dfn4.Survey = dfn4.index
dfn4.index = np.arange(len(dfn4))

dict_names = {}
second  = dfn4.iloc[0].fillna("")
for i in range(len(dfn4.columns)):
    first = dfn4.columns[i]
    if first.startswith("Unnamed"):
        if dfn4.columns[i-1].startswith("Unnamed"):
            first = dfn4.columns[i+1]
        else:
            first = dfn4.columns[i-1]
    #         new = first+second[i]
    # else:
    new = first+" "+second[i]
    dict_names[dfn4.columns[i]]= new
dfn4 = dfn4.rename(columns = dict_names).drop([0])

# show the first five records
dfn4.head()

,Survey Date,State Code,6-Digit ID,County Code,Census Place Code,FIPS Place Code,FIPS MCD Code,Pop,CSA Code,CBSA Code,Footnote Code,Central City,Zip Code,Region Code,Division Code,Number of Months Rep,Place Name,Place Bldgs,1-unit Units,1-unit Value,2-units Bldgs,2-units Units,2-units Value,3-4 units Bldgs,3-4 units Units,3-4 units Value,5+ units Bldgs,5+ units Units,5+ units Value,1-unit rep Bldgs,1-unit rep Units,1-unit rep Value,2-units rep Bldgs,2-units rep Units,2-units rep Value,3-4 units rep Bldgs,3-4 units rep Units,3-4 units rep Value,5+ units rep Bldgs,5+ units rep Units,5+ units rep Value
1,2018,09,001000,013,NaN,00000,01080,3303.0,278,25540,NaN,NaN,06232,1,1,12,Andover town,5,5,973000,0,0,0,0,0,0,0,0,0,5,5,973000,0,0,0,0,0,0,0,0,0
2,2018,09,005000,009,0010,01150,01220,19249.0,408,35300,2,NaN,064011855,1,1,12,Ansonia,2,2,300000,1,2,150000,0,0,0,0,0,0,2,2,300000,1,2,150000,0,0,0,0,0,0
3,2018,09,009000,015,NaN,00000,01430,4317.0,148,49340,NaN,NaN,06278,1,1,12,Ashford town,9,9,1945000,0,0,0,0,0,0,0,0,0,9,9,1945000,0,0,0,0,0,0,0,0,0
4,2018,09,013000,003,NaN,00000,02060,18098.0,278,25540,NaN,NaN,06001,1,1,12,Avon town,13,13,3163450,0,0,0,0,0,0,0,0,0,13,13,3163450,0,0,0,0,0,0,0,0,0
5,2018,09,021000,005,NaN,00000,02760,3799.0,999,99999,NaN,NaN,060633340,1,1,12,Barkhamsted town,2,2,552000,0,0,0,0,0,0,0,0,0,2,2,552000,0,0,0,0,0,0,0,0,0


In [29]:
dfn5 = pd.read_table("https://www2.census.gov/econ/bps/Place/Northeast%20Region/ne2019a.txt", sep=",",
                  skipinitialspace=True) #read the online table

# the web data table has some format issue, this block of code is to address that issue
dfn5["Unnamed 41"] = np.nan
dfn5 = dfn5.shift(1, axis=1)
dfn5.Survey = dfn5.index
dfn5.index = np.arange(len(dfn5))

dict_names = {}
second  = dfn5.iloc[0].fillna("")
for i in range(len(dfn5.columns)):
    first = dfn5.columns[i]
    if first.startswith("Unnamed"):
        if dfn5.columns[i-1].startswith("Unnamed"):
            first = dfn5.columns[i+1]
        else:
            first = dfn5.columns[i-1]
    #         new = first+second[i]
    # else:
    new = first+" "+second[i]
    dict_names[dfn5.columns[i]]= new
dfn5 = dfn5.rename(columns = dict_names).drop([0])

# show the first five records
dfn5.head()

,Survey Date,State Code,6-Digit ID,County Code,Census Place Code,FIPS Place Code,FIPS MCD Code,Pop,CSA Code,CBSA Code,Footnote Code,Central City,Zip Code,Region Code,Division Code,Number of Months Rep,Place Name,Place Bldgs,1-unit Units,1-unit Value,2-units Bldgs,2-units Units,2-units Value,3-4 units Bldgs,3-4 units Units,3-4 units Value,5+ units Bldgs,5+ units Units,5+ units Value,1-unit rep Bldgs,1-unit rep Units,1-unit rep Value,2-units rep Bldgs,2-units rep Units,2-units rep Value,3-4 units rep Bldgs,3-4 units rep Units,3-4 units rep Value,5+ units rep Bldgs,5+ units rep Units,5+ units rep Value
1,2019,09,001000,013,NaN,00000,01080,3303.0,278,25540,NaN,NaN,06232,1,1,12,Andover town,7,7,1673875,0,0,0,0,0,0,0,0,0,7,7,1673875,0,0,0,0,0,0,0,0,0
2,2019,09,005000,009,0010,01150,01220,19249.0,408,35300,2,NaN,064011855,1,1,12,Ansonia,1,1,90000,0,0,0,0,0,0,0,0,0,1,1,90000,0,0,0,0,0,0,0,0,0
3,2019,09,009000,015,NaN,00000,01430,4317.0,148,49340,NaN,NaN,06278,1,1,12,Ashford town,5,5,1054500,0,0,0,0,0,0,0,0,0,5,5,1054500,0,0,0,0,0,0,0,0,0
4,2019,09,013000,003,NaN,00000,02060,18098.0,278,25540,NaN,NaN,06001,1,1,12,Avon town,13,13,3754638,0,0,0,0,0,0,0,0,0,13,13,3754638,0,0,0,0,0,0,0,0,0
5,2019,09,021000,005,NaN,00000,02760,3799.0,999,99999,NaN,NaN,060633340,1,1,12,Barkhamsted town,3,3,924380,0,0,0,0,0,0,0,0,0,3,3,924380,0,0,0,0,0,0,0,0,0


In [30]:
vertical_concatne = pd.concat([dfn1,dfn2, dfn3,dfn4,dfn5], axis=0)

In [31]:
dfne=vertical_concatne

In [32]:
dfne

,Survey Date,State Code,6-Digit ID,County Code,Census Place Code,FIPS Place Code,FIPS MCD Code,Pop,CSA Code,CBSA Code,Footnote Code,Central City,Zip Code,Region Code,Division Code,Number of Months Rep,Place Name,Place Bldgs,1-unit Units,1-unit Value,2-units Bldgs,2-units Units,2-units Value,3-4 units Bldgs,3-4 units Units,3-4 units Value,5+ units Bldgs,5+ units Units,5+ units Value,1-unit rep Bldgs,1-unit rep Units,1-unit rep Value,2-units rep Bldgs,2-units rep Units,2-units rep Value,3-4 units rep Bldgs,3-4 units rep Units,3-4 units rep Value,5+ units rep Bldgs,5+ units rep Units,5+ units rep Value
1,2015,09,001000,013,NaN,00000,01080,3303.0,278,25540,NaN,NaN,06232,1,1,12,Andover town,4,4,754900,0,0,0,0,0,0,0,0,0,4,4,754900,0,0,0,0,0,0,0,0,0
2,2015,09,005000,009,0010,01150,01220,19249.0,408,35300,2,NaN,064011855,1,1,0,Ansonia,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,2015,09,009000,015,NaN,00000,01430,4317.0,148,49340,NaN,NaN,06278,1,1,12,Ashford town,2,2,281500,0,0,0,0,0,0,0,0,0,2,2,281500,0,0,0,0,0,0,0,0,0
4,2015,09,013000,003,NaN,00000,02060,18098.0,278,25540,NaN,NaN,06001,1,1,12,Avon town,31,31,10995334,0,0,0,0,0,0,0,0,0,31,31,10995334,0,0,0,0,0,0,0,0,0
5,2015,09,021000,005,NaN,00000,02760,3799.0,999,99999,NaN,NaN,060633340,1,1,12,Barkhamsted town,3,3,1035150,0,0,0,0,0,0,0,0,0,3,3,1035150,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5572,2019,50,591000,015,NaN,00000,85375,1676.0,999,99999,NaN,NaN,05680,1,1,12,Wolcott town,5,5,220000,0,0,0,0,0,0,0,0,0,5,5,220000,0,0,0,0,0,0,0,0,0
5573,2019,50,593000,023,NaN,00000,85525,906.0,999,99999,NaN,NaN,05683,1,1,0,Woodbury town,4,4,984063,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5574,2019,50,595000,003,NaN,00000,85675,424.0,999,99999,NaN,NaN,052019410,1,1,12,Woodford town,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5575,2019,50,597000,027,NaN,00000,85975,2148.0,999,99999,NaN,NaN,05091,1,1,12,Woodstock town,4,4,1128930,0,0,0,0,0,0,0,0,0,4,4,1128930,0,0,0,0,0,0,0,0,0


In [33]:
dfne=dfne.reset_index()

In [34]:
# Drop a specific column, e.g., 'Place Name'
dfne = dfne.drop(columns=['index'])

In [35]:
dfne

,Survey Date,State Code,6-Digit ID,County Code,Census Place Code,FIPS Place Code,FIPS MCD Code,Pop,CSA Code,CBSA Code,Footnote Code,Central City,Zip Code,Region Code,Division Code,Number of Months Rep,Place Name,Place Bldgs,1-unit Units,1-unit Value,2-units Bldgs,2-units Units,2-units Value,3-4 units Bldgs,3-4 units Units,3-4 units Value,5+ units Bldgs,5+ units Units,5+ units Value,1-unit rep Bldgs,1-unit rep Units,1-unit rep Value,2-units rep Bldgs,2-units rep Units,2-units rep Value,3-4 units rep Bldgs,3-4 units rep Units,3-4 units rep Value,5+ units rep Bldgs,5+ units rep Units,5+ units rep Value
0,2015,09,001000,013,NaN,00000,01080,3303.0,278,25540,NaN,NaN,06232,1,1,12,Andover town,4,4,754900,0,0,0,0,0,0,0,0,0,4,4,754900,0,0,0,0,0,0,0,0,0
1,2015,09,005000,009,0010,01150,01220,19249.0,408,35300,2,NaN,064011855,1,1,0,Ansonia,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2015,09,009000,015,NaN,00000,01430,4317.0,148,49340,NaN,NaN,06278,1,1,12,Ashford town,2,2,281500,0,0,0,0,0,0,0,0,0,2,2,281500,0,0,0,0,0,0,0,0,0
3,2015,09,013000,003,NaN,00000,02060,18098.0,278,25540,NaN,NaN,06001,1,1,12,Avon town,31,31,10995334,0,0,0,0,0,0,0,0,0,31,31,10995334,0,0,0,0,0,0,0,0,0
4,2015,09,021000,005,NaN,00000,02760,3799.0,999,99999,NaN,NaN,060633340,1,1,12,Barkhamsted town,3,3,1035150,0,0,0,0,0,0,0,0,0,3,3,1035150,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27887,2019,50,591000,015,NaN,00000,85375,1676.0,999,99999,NaN,NaN,05680,1,1,12,Wolcott town,5,5,220000,0,0,0,0,0,0,0,0,0,5,5,220000,0,0,0,0,0,0,0,0,0
27888,2019,50,593000,023,NaN,00000,85525,906.0,999,99999,NaN,NaN,05683,1,1,0,Woodbury town,4,4,984063,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
27889,2019,50,595000,003,NaN,00000,85675,424.0,999,99999,NaN,NaN,052019410,1,1,12,Woodford town,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
27890,2019,50,597000,027,NaN,00000,85975,2148.0,999,99999,NaN,NaN,05091,1,1,12,Woodstock town,4,4,1128930,0,0,0,0,0,0,0,0,0,4,4,1128930,0,0,0,0,0,0,0,0,0


In [36]:
import pandas as pd

# Assuming your data is in a CSV file named 'data.csv'
datane = dfne

# Select the columns of interest
selected_columnsne = ['FIPS Place Code','Zip Code','Survey Date', '1-unit rep Units', 'Place Name']

# Extract the desired data
extracted_datasne = datane[selected_columnsne]


In [37]:
#choosing the column with 1-unit rep unit data
extracted_datasne

,FIPS Place Code,Zip Code,Survey Date,1-unit rep Units,Place Name
0,00000,06232,2015,4,Andover town
1,01150,064011855,2015,0,Ansonia
2,00000,06278,2015,2,Ashford town
3,00000,06001,2015,31,Avon town
4,00000,060633340,2015,3,Barkhamsted town
...,...,...,...,...,...
27887,00000,05680,2019,5,Wolcott town
27888,00000,05683,2019,0,Woodbury town
27889,00000,052019410,2019,0,Woodford town
27890,00000,05091,2019,4,Woodstock town


In [38]:
# Group by 'Zip Code', 'FIPS Place Code', and 'Place Name', and sum '1-unit rep Units'
grouped_df1ne = extracted_datasne.groupby(['Zip Code', 'FIPS Place Code', 'Place Name', 'Survey Date'])['1-unit rep Units'].sum().reset_index()

# Pivot the DataFrame to have years as columns
pivot_df1ne = grouped_df1ne.pivot_table(index=['Zip Code', 'FIPS Place Code', 'Place Name'], columns='Survey Date', values='1-unit rep Units', fill_value=0)

# Reset the index to have 'Zip Code', 'FIPS Place Code', and 'Place Name' as columns
pivot_df1ne.reset_index(inplace=True)

# Rename the columns
pivot_df1ne.columns = ['Zip Code', 'FIPS Place Code', 'Place Name', '2015', '2016', '2017', '2018', '2019']


# Display the pivot DataFrame
print(pivot_df1ne)

         Zip Code FIPS Place Code                 Place Name  2015  2016  \
0     01001                00840                 Agawam Town    12    16   
1     01002                00000                Amherst town     0     0   
2     01002                00000                 Pelham town     0     1   
3     01005                00000                  Barre town     7    14   
4     01007                00000            Belchertown town    35    65   
...           ...             ...                        ...   ...   ...   
5970  19608                00000   South Heidelberg township     0     2   
5971  19608                00000             Spring township    12    13   
5972  19608                70880      Sinking Spring borough     0     0   
5973  19610                86880          Wyomissing borough     1     0   
5974  19611                83928        West Reading borough     0     0   

      2017  2018  2019  
0       31    19     8  
1        0    25     0  
2        2  

In [39]:
pivot_df1ne

,Zip Code,FIPS Place Code,Place Name,2015,2016,2017,2018,2019
0,01001,00840,Agawam Town,12,16,31,19,8
1,01002,00000,Amherst town,0,0,0,25,0
2,01002,00000,Pelham town,0,1,2,1,1
3,01005,00000,Barre town,7,14,7,10,9
4,01007,00000,Belchertown town,35,65,39,58,41
...,...,...,...,...,...,...,...,...
5970,19608,00000,South Heidelberg township,0,2,3,6,17
5971,19608,00000,Spring township,12,13,2,12,6
5972,19608,70880,Sinking Spring borough,0,0,0,0,1
5973,19610,86880,Wyomissing borough,1,0,1,3,1


In [40]:
dfne

,Survey Date,State Code,6-Digit ID,County Code,Census Place Code,FIPS Place Code,FIPS MCD Code,Pop,CSA Code,CBSA Code,Footnote Code,Central City,Zip Code,Region Code,Division Code,Number of Months Rep,Place Name,Place Bldgs,1-unit Units,1-unit Value,2-units Bldgs,2-units Units,2-units Value,3-4 units Bldgs,3-4 units Units,3-4 units Value,5+ units Bldgs,5+ units Units,5+ units Value,1-unit rep Bldgs,1-unit rep Units,1-unit rep Value,2-units rep Bldgs,2-units rep Units,2-units rep Value,3-4 units rep Bldgs,3-4 units rep Units,3-4 units rep Value,5+ units rep Bldgs,5+ units rep Units,5+ units rep Value
0,2015,09,001000,013,NaN,00000,01080,3303.0,278,25540,NaN,NaN,06232,1,1,12,Andover town,4,4,754900,0,0,0,0,0,0,0,0,0,4,4,754900,0,0,0,0,0,0,0,0,0
1,2015,09,005000,009,0010,01150,01220,19249.0,408,35300,2,NaN,064011855,1,1,0,Ansonia,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2015,09,009000,015,NaN,00000,01430,4317.0,148,49340,NaN,NaN,06278,1,1,12,Ashford town,2,2,281500,0,0,0,0,0,0,0,0,0,2,2,281500,0,0,0,0,0,0,0,0,0
3,2015,09,013000,003,NaN,00000,02060,18098.0,278,25540,NaN,NaN,06001,1,1,12,Avon town,31,31,10995334,0,0,0,0,0,0,0,0,0,31,31,10995334,0,0,0,0,0,0,0,0,0
4,2015,09,021000,005,NaN,00000,02760,3799.0,999,99999,NaN,NaN,060633340,1,1,12,Barkhamsted town,3,3,1035150,0,0,0,0,0,0,0,0,0,3,3,1035150,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27887,2019,50,591000,015,NaN,00000,85375,1676.0,999,99999,NaN,NaN,05680,1,1,12,Wolcott town,5,5,220000,0,0,0,0,0,0,0,0,0,5,5,220000,0,0,0,0,0,0,0,0,0
27888,2019,50,593000,023,NaN,00000,85525,906.0,999,99999,NaN,NaN,05683,1,1,0,Woodbury town,4,4,984063,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
27889,2019,50,595000,003,NaN,00000,85675,424.0,999,99999,NaN,NaN,052019410,1,1,12,Woodford town,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
27890,2019,50,597000,027,NaN,00000,85975,2148.0,999,99999,NaN,NaN,05091,1,1,12,Woodstock town,4,4,1128930,0,0,0,0,0,0,0,0,0,4,4,1128930,0,0,0,0,0,0,0,0,0


In [41]:
import pandas as pd

# Assuming your data is in a CSV file named 'data.csv'
datane2 = dfne

# Select the columns of interest
selected_columnsne2 = ['FIPS Place Code','Zip Code','Survey Date', '2-units rep Units','3-4 units rep Units','5+ units rep Units', 'Place Name']

# Extract the desired data
extracted_datamne2 = datane2[selected_columnsne2]

In [42]:
extracted_datamne2

,FIPS Place Code,Zip Code,Survey Date,2-units rep Units,3-4 units rep Units,5+ units rep Units,Place Name
0,00000,06232,2015,0,0,0,Andover town
1,01150,064011855,2015,0,0,0,Ansonia
2,00000,06278,2015,0,0,0,Ashford town
3,00000,06001,2015,0,0,0,Avon town
4,00000,060633340,2015,0,0,0,Barkhamsted town
...,...,...,...,...,...,...,...
27887,00000,05680,2019,0,0,0,Wolcott town
27888,00000,05683,2019,0,0,0,Woodbury town
27889,00000,052019410,2019,0,0,0,Woodford town
27890,00000,05091,2019,0,0,0,Woodstock town


In [43]:
# Group by 'Zip Code,' 'FIPS Place Code,' 'Place Name,' and 'Survey Date,' and sum the units columns
grouped_dfmne = extracted_datamne2.groupby(['Zip Code', 'FIPS Place Code', 'Place Name', 'Survey Date']).agg({
    '2-units rep Units': 'sum',
    '3-4 units rep Units': 'sum',
    '5+ units rep Units': 'sum'
}).reset_index()

# Create a new column 'multi unit' by summing the '2-units rep Units,' '3-4 units rep Units,' and '5+ units rep Units'
grouped_dfmne['multi unit'] = grouped_dfmne['2-units rep Units'] + grouped_dfmne['3-4 units rep Units'] + grouped_dfmne['5+ units rep Units']


# Display the grouped DataFrame
print(grouped_dfmne)





          Zip Code FIPS Place Code            Place Name Survey Date  \
0      01001                00840            Agawam Town        2015   
1      01001                00840            Agawam Town        2016   
2      01001                00840            Agawam Town        2017   
3      01001                00840            Agawam Town        2018   
4      01001                00840            Agawam Town        2019   
...            ...             ...                   ...         ...   
27850  19611                83928   West Reading borough        2015   
27851  19611                83928   West Reading borough        2016   
27852  19611                83928   West Reading borough        2017   
27853  19611                83928   West Reading borough        2018   
27854  19611                83928   West Reading borough        2019   

      2-units rep Units 3-4 units rep Units 5+ units rep Units multi unit  
0                    20                   0                

In [44]:
grouped_dfmne

,Zip Code,FIPS Place Code,Place Name,Survey Date,2-units rep Units,3-4 units rep Units,5+ units rep Units,multi unit
0,01001,00840,Agawam Town,2015,20,0,0,2000
1,01001,00840,Agawam Town,2016,8,3,16,8316
2,01001,00840,Agawam Town,2017,2,0,0,200
3,01001,00840,Agawam Town,2018,4,0,0,400
4,01001,00840,Agawam Town,2019,6,0,0,600
...,...,...,...,...,...,...,...,...
27850,19611,83928,West Reading borough,2015,0,0,0,000
27851,19611,83928,West Reading borough,2016,0,0,0,000
27852,19611,83928,West Reading borough,2017,0,0,0,000
27853,19611,83928,West Reading borough,2018,0,0,0,000


In [45]:
# Pivot the DataFrame to have years as columns
pivot_dfmne = grouped_dfmne.pivot_table(index=['Zip Code', 'FIPS Place Code', 'Place Name'], columns='Survey Date', values='multi unit', fill_value=0)

# Reset the index to have 'Zip Code', 'FIPS Place Code', and 'Place Name' as columns
pivot_dfmne.reset_index(inplace=True)

# Rename the columns
pivot_dfmne.columns = ['Zip Code', 'FIPS Place Code', 'Place Name', '2015', '2016', '2017', '2018', '2019']


# Display the pivot DataFrame
print(pivot_dfmne)

         Zip Code FIPS Place Code                 Place Name  2015  2016  \
0     01001                00840                 Agawam Town  2000  8316   
1     01002                00000                Amherst town     0     0   
2     01002                00000                 Pelham town     0     0   
3     01005                00000                  Barre town   200   200   
4     01007                00000            Belchertown town     0     0   
...           ...             ...                        ...   ...   ...   
5970  19608                00000   South Heidelberg township     0     0   
5971  19608                00000             Spring township     0     0   
5972  19608                70880      Sinking Spring borough     0     0   
5973  19610                86880          Wyomissing borough     0     0   
5974  19611                83928        West Reading borough     0     0   

      2017   2018  2019  
0      200    400   600  
1        0  80126     0  
2        

In [46]:
pivot_dfmne

,Zip Code,FIPS Place Code,Place Name,2015,2016,2017,2018,2019
0,01001,00840,Agawam Town,2000,8316,200,400,600
1,01002,00000,Amherst town,0,0,0,80126,0
2,01002,00000,Pelham town,0,0,0,0,0
3,01005,00000,Barre town,200,200,0,0,0
4,01007,00000,Belchertown town,0,0,0,0,0
...,...,...,...,...,...,...,...,...
5970,19608,00000,South Heidelberg township,0,0,0,0,0
5971,19608,00000,Spring township,0,0,400,0,210
5972,19608,70880,Sinking Spring borough,0,0,0,0,0
5973,19610,86880,Wyomissing borough,0,0,0,84,0


In [47]:
#extra
import pandas as pd

# Assuming you have the 'grouped_dfmne' DataFrame

# List of unique survey years in the dataset
survey_years = ['2015', '2016', '2017', '2018', '2019']

# Initialize an empty DataFrame to store the pivoted data
pivoted_dfsne = []

# Loop through each survey year and pivot the DataFrame
for year in survey_years:
    # Filter the DataFrame for the current year
    filtered_dfne = grouped_dfmne[grouped_dfmne['Survey Date'] == year]

    # Pivot the filtered DataFrame
    pivot_dfne = filtered_dfne.pivot_table(index=['Zip Code', 'FIPS Place Code', 'Place Name'],
                                       columns='Survey Date', values='multi unit', fill_value=0)

    # Reset the index to have 'Zip Code', 'FIPS Place Code', and 'Place Name' as columns
    pivot_dfne.reset_index(inplace=True)

    # Rename the columns
    pivot_dfne.columns = ['Zip Code', 'FIPS Place Code', 'Place Name', year]

    # Append the pivoted DataFrame to the list
    pivoted_dfsne.append(pivot_dfne)

# Merge the individual pivoted DataFrames on 'Zip Code', 'FIPS Place Code', and 'Place Name'
final_pivot_dfne = pivoted_dfsne[0]
for i in range(1, len(pivoted_dfsne)):
    final_pivot_dfne = final_pivot_dfne.merge(pivoted_dfsne[i], on=['Zip Code', 'FIPS Place Code', 'Place Name'], how='outer')


# Display the final pivoted DataFrame
print(final_pivot_dfne)


         Zip Code FIPS Place Code            Place Name    2015    2016  \
0     01001                00840            Agawam Town  2000.0  8316.0   
1     01002                00000           Amherst town     0.0     0.0   
2     01002                00000            Pelham town     0.0     0.0   
3     01005                00000             Barre town   200.0   200.0   
4     01007                00000       Belchertown town     0.0     0.0   
...           ...             ...                   ...     ...     ...   
5970  18801                00000   Forest Lake township     NaN     NaN   
5971  18854                00000         Wysox township     NaN     NaN   
5972  19018                15432         Colwyn borough     NaN     NaN   
5973  193204950            00000   Londonderry township     NaN     NaN   
5974  19526                00000   Charlestown township     NaN     NaN   

       2017     2018   2019  
0     200.0    400.0  600.0  
1       0.0  80126.0    0.0  
2       0

In [48]:
#extra
final_pivot_dfne

,Zip Code,FIPS Place Code,Place Name,2015,2016,2017,2018,2019
0,01001,00840,Agawam Town,2000.0,8316.0,200.0,400.0,600.0
1,01002,00000,Amherst town,0.0,0.0,0.0,80126.0,0.0
2,01002,00000,Pelham town,0.0,0.0,0.0,0.0,0.0
3,01005,00000,Barre town,200.0,200.0,0.0,0.0,0.0
4,01007,00000,Belchertown town,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...
5970,18801,00000,Forest Lake township,NaN,NaN,NaN,NaN,0.0
5971,18854,00000,Wysox township,NaN,NaN,NaN,NaN,0.0
5972,19018,15432,Colwyn borough,NaN,NaN,NaN,NaN,0.0
5973,193204950,00000,Londonderry township,NaN,NaN,NaN,NaN,400.0


# South region

# **Repeating all steps for south region**

In [49]:
#https://www2.census.gov/econ/bps/Place/Northeast%20Region/ne2015a.txt

In [50]:
dfso1 = pd.read_table("https://www2.census.gov/econ/bps/Place/South%20Region/so2015a.txt", sep=",",
                  skipinitialspace=True) #read the online table

# the web data table has some format issue, this block of code is to address that issue
dfso1["Unnamed 41"] = np.nan
dfso1 = dfso1.shift(1, axis=1)
dfso1.Survey = dfso1.index
dfso1.index = np.arange(len(dfso1))

dict_names = {}
second  = dfso1.iloc[0].fillna("")
for i in range(len(dfso1.columns)):
    first = dfso1.columns[i]
    if first.startswith("Unnamed"):
        if dfso1.columns[i-1].startswith("Unnamed"):
            first = dfso1.columns[i+1]
        else:
            first = dfso1.columns[i-1]
    #         new = first+second[i]
    # else:
    new = first+" "+second[i]
    dict_names[dfso1.columns[i]]= new
dfso1 = dfso1.rename(columns = dict_names).drop([0])

# show the first five records
dfso1.head()

,Survey Date,State Code,6-Digit ID,County Code,Census Place Code,FIPS Place Code,FIPS MCD Code,Pop,CSA Code,CBSA Code,Footnote Code,Central City,Zip Code,Region Code,Division Code,Number of Months Rep,Place Name,Place Bldgs,1-unit Units,1-unit Value,2-units Bldgs,2-units Units,2-units Value,3-4 units Bldgs,3-4 units Units,3-4 units Value,5+ units Bldgs,5+ units Units,5+ units Value,1-unit rep Bldgs,1-unit rep Units,1-unit rep Value,2-units rep Bldgs,2-units rep Units,2-units rep Value,3-4 units rep Bldgs,3-4 units rep Units,3-4 units rep Value,5+ units rep Bldgs,5+ units rep Units,5+ units rep Value
1,2015,01,001000,067,0020,00124,00000,2688.0,222,20020,NaN,NaN,36310,3,6,12,Abbeville,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2015,01,002000,073,0030,00460,00000,4522.0,142,13820,NaN,NaN,35005,3,6,12,Adamsville,1,1,200596,0,0,0,0,0,0,0,0,0,1,1,200596,0,0,0,0,0,0,0,0,0
3,2015,01,005000,117,0025,00820,00000,30352.0,142,13820,NaN,NaN,35007,3,6,12,Alabaster,109,109,33075811,0,0,0,0,0,0,0,0,0,109,109,33075811,0,0,0,0,0,0,0,0,0
4,2015,01,006000,095,0030,00988,00000,21160.0,999,99999,NaN,NaN,35950,3,6,12,Albertville,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,2015,01,007000,123,0035,01132,00000,14875.0,999,99999,NaN,NaN,350110552,3,6,12,Alexander City,18,18,4124433,0,0,0,0,0,0,0,0,0,18,18,4124433,0,0,0,0,0,0,0,0,0


In [51]:
dfso2 = pd.read_table("https://www2.census.gov/econ/bps/Place/South%20Region/so2016a.txt", sep=",",
                  skipinitialspace=True) #read the online table

# the web data table has some format issue, this block of code is to address that issue
dfso2["Unnamed 41"] = np.nan
dfso2 = dfso2.shift(1, axis=1)
dfso2.Survey = dfso2.index
dfso2.index = np.arange(len(dfso2))

dict_names = {}
second  = dfso2.iloc[0].fillna("")
for i in range(len(dfso2.columns)):
    first = dfso2.columns[i]
    if first.startswith("Unnamed"):
        if dfso2.columns[i-1].startswith("Unnamed"):
            first = dfso2.columns[i+1]
        else:
            first = dfso2.columns[i-1]
    #         new = first+second[i]
    # else:
    new = first+" "+second[i]
    dict_names[dfso2.columns[i]]= new
dfso2 = dfso2.rename(columns = dict_names).drop([0])

# show the first five records
dfso2.head()

,Survey Date,State Code,6-Digit ID,County Code,Census Place Code,FIPS Place Code,FIPS MCD Code,Pop,CSA Code,CBSA Code,Footnote Code,Central City,Zip Code,Region Code,Division Code,Number of Months Rep,Place Name,Place Bldgs,1-unit Units,1-unit Value,2-units Bldgs,2-units Units,2-units Value,3-4 units Bldgs,3-4 units Units,3-4 units Value,5+ units Bldgs,5+ units Units,5+ units Value,1-unit rep Bldgs,1-unit rep Units,1-unit rep Value,2-units rep Bldgs,2-units rep Units,2-units rep Value,3-4 units rep Bldgs,3-4 units rep Units,3-4 units rep Value,5+ units rep Bldgs,5+ units rep Units,5+ units rep Value
1,2016,01,001000,067,0020,00124,00000,2688.0,222,20020,NaN,NaN,36310,3,6,0,Abbeville,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2016,01,002000,073,0030,00460,00000,4522.0,142,13820,NaN,NaN,35005,3,6,12,Adamsville,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,2016,01,005000,117,0025,00820,00000,30352.0,142,13820,NaN,NaN,35007,3,6,12,Alabaster,132,132,32974365,0,0,0,0,0,0,0,0,0,132,132,32974365,0,0,0,0,0,0,0,0,0
4,2016,01,006000,095,0030,00988,00000,21160.0,999,99999,NaN,NaN,35950,3,6,12,Albertville,2,2,404000,0,0,0,0,0,0,0,0,0,2,2,404000,0,0,0,0,0,0,0,0,0
5,2016,01,007000,123,0035,01132,00000,14875.0,999,99999,NaN,NaN,350110552,3,6,12,Alexander City,12,12,3607770,0,0,0,0,0,0,0,0,0,12,12,3607770,0,0,0,0,0,0,0,0,0


In [52]:
dfso3 = pd.read_table("https://www2.census.gov/econ/bps/Place/South%20Region/so2017a.txt", sep=",",
                  skipinitialspace=True) #read the online table

# the web data table has some format issue, this block of code is to address that issue
dfso3["Unnamed 41"] = np.nan
dfso3 = dfso3.shift(1, axis=1)
dfso3.Survey = dfso3.index
dfso3.index = np.arange(len(dfso3))

dict_names = {}
second  = dfso3.iloc[0].fillna("")
for i in range(len(dfso3.columns)):
    first = dfso3.columns[i]
    if first.startswith("Unnamed"):
        if dfso3.columns[i-1].startswith("Unnamed"):
            first = dfso3.columns[i+1]
        else:
            first = dfso3.columns[i-1]
    #         new = first+second[i]
    # else:
    new = first+" "+second[i]
    dict_names[dfso3.columns[i]]= new
dfso3 = dfso3.rename(columns = dict_names).drop([0])

# show the first five records
dfso3.head()

,Survey Date,State Code,6-Digit ID,County Code,Census Place Code,FIPS Place Code,FIPS MCD Code,Pop,CSA Code,CBSA Code,Footnote Code,Central City,Zip Code,Region Code,Division Code,Number of Months Rep,Place Name,Place Bldgs,1-unit Units,1-unit Value,2-units Bldgs,2-units Units,2-units Value,3-4 units Bldgs,3-4 units Units,3-4 units Value,5+ units Bldgs,5+ units Units,5+ units Value,1-unit rep Bldgs,1-unit rep Units,1-unit rep Value,2-units rep Bldgs,2-units rep Units,2-units rep Value,3-4 units rep Bldgs,3-4 units rep Units,3-4 units rep Value,5+ units rep Bldgs,5+ units rep Units,5+ units rep Value
1,2017,01,001000,067,0020,00124,00000,2688.0,222,20020,NaN,NaN,36310,3,6,12,Abbeville,1,1,350000,0,0,0,0,0,0,0,0,0,1,1,350000,0,0,0,0,0,0,0,0,0
2,2017,01,002000,073,0030,00460,00000,4522.0,142,13820,NaN,NaN,35005,3,6,12,Adamsville,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,2017,01,005000,117,0025,00820,00000,30352.0,142,13820,NaN,NaN,35007,3,6,12,Alabaster,92,92,22168266,0,0,0,0,0,0,0,0,0,92,92,22168266,0,0,0,0,0,0,0,0,0
4,2017,01,006000,095,0030,00988,00000,21160.0,999,99999,NaN,NaN,35950,3,6,12,Albertville,23,23,3707000,0,0,0,0,0,0,0,0,0,23,23,3707000,0,0,0,0,0,0,0,0,0
5,2017,01,007000,123,0035,01132,00000,14875.0,999,99999,NaN,NaN,350110552,3,6,12,Alexander City,14,14,3247000,0,0,0,0,0,0,0,0,0,14,14,3247000,0,0,0,0,0,0,0,0,0


In [53]:
dfso4 = pd.read_table("https://www2.census.gov/econ/bps/Place/South%20Region/so2018a.txt", sep=",",
                  skipinitialspace=True) #read the online table

# the web data table has some format issue, this block of code is to address that issue
dfso4["Unnamed 41"] = np.nan
dfso4 = dfso4.shift(1, axis=1)
dfso4.Survey = dfso4.index
dfso4.index = np.arange(len(dfso4))

dict_names = {}
second  = dfso4.iloc[0].fillna("")
for i in range(len(dfso4.columns)):
    first = dfso4.columns[i]
    if first.startswith("Unnamed"):
        if dfso4.columns[i-1].startswith("Unnamed"):
            first = dfso4.columns[i+1]
        else:
            first = dfso4.columns[i-1]
    #         new = first+second[i]
    # else:
    new = first+" "+second[i]
    dict_names[dfso4.columns[i]]= new
dfso4 = dfso4.rename(columns = dict_names).drop([0])

# show the first five records
dfso4.head()

,Survey Date,State Code,6-Digit ID,County Code,Census Place Code,FIPS Place Code,FIPS MCD Code,Pop,CSA Code,CBSA Code,Footnote Code,Central City,Zip Code,Region Code,Division Code,Number of Months Rep,Place Name,Place Bldgs,1-unit Units,1-unit Value,2-units Bldgs,2-units Units,2-units Value,3-4 units Bldgs,3-4 units Units,3-4 units Value,5+ units Bldgs,5+ units Units,5+ units Value,1-unit rep Bldgs,1-unit rep Units,1-unit rep Value,2-units rep Bldgs,2-units rep Units,2-units rep Value,3-4 units rep Bldgs,3-4 units rep Units,3-4 units rep Value,5+ units rep Bldgs,5+ units rep Units,5+ units rep Value
1,2018,01,001000,067,0020,00124,00000,2688.0,222,20020,NaN,NaN,36310,3,6,12,Abbeville,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2018,01,002000,073,0030,00460,00000,4522.0,142,13820,NaN,NaN,35005,3,6,8,Adamsville,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,2018,01,005000,117,0025,00820,00000,30352.0,142,13820,NaN,NaN,35007,3,6,10,Alabaster,65,65,19967174,0,0,0,0,0,0,0,0,0,52,52,16834702,0,0,0,0,0,0,0,0,0
4,2018,01,006000,095,0030,00988,00000,21160.0,999,99999,NaN,NaN,35950,3,6,12,Albertville,34,34,4997000,0,0,0,0,0,0,0,0,0,34,34,4997000,0,0,0,0,0,0,0,0,0
5,2018,01,007000,123,0035,01132,00000,14875.0,999,99999,NaN,NaN,350110552,3,6,12,Alexander City,13,13,3177319,0,0,0,0,0,0,0,0,0,13,13,3177319,0,0,0,0,0,0,0,0,0


In [54]:
dfso5 = pd.read_table("https://www2.census.gov/econ/bps/Place/South%20Region/so2019a.txt", sep=",",
                  skipinitialspace=True) #read the online table

# the web data table has some format issue, this block of code is to address that issue
dfso5["Unnamed 41"] = np.nan
dfso5 = dfso5.shift(1, axis=1)
dfso5.Survey = dfso5.index
dfso5.index = np.arange(len(dfso5))

dict_names = {}
second  = dfso5.iloc[0].fillna("")
for i in range(len(dfso5.columns)):
    first = dfso5.columns[i]
    if first.startswith("Unnamed"):
        if dfso5.columns[i-1].startswith("Unnamed"):
            first = dfso5.columns[i+1]
        else:
            first = dfso5.columns[i-1]
    #         new = first+second[i]
    # else:
    new = first+" "+second[i]
    dict_names[dfso5.columns[i]]= new
dfso5 = dfso5.rename(columns = dict_names).drop([0])

# show the first five records
dfso5.head()

,Survey Date,State Code,6-Digit ID,County Code,Census Place Code,FIPS Place Code,FIPS MCD Code,Pop,CSA Code,CBSA Code,Footnote Code,Central City,Zip Code,Region Code,Division Code,Number of Months Rep,Place Name,Place Bldgs,1-unit Units,1-unit Value,2-units Bldgs,2-units Units,2-units Value,3-4 units Bldgs,3-4 units Units,3-4 units Value,5+ units Bldgs,5+ units Units,5+ units Value,1-unit rep Bldgs,1-unit rep Units,1-unit rep Value,2-units rep Bldgs,2-units rep Units,2-units rep Value,3-4 units rep Bldgs,3-4 units rep Units,3-4 units rep Value,5+ units rep Bldgs,5+ units rep Units,5+ units rep Value
1,2019,01,001000,067,0020,00124,00000,2688.0,222,20020,NaN,NaN,36310,3,6,12,Abbeville,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2019,01,002000,073,0030,00460,00000,4522.0,142,13820,NaN,NaN,35005,3,6,12,Adamsville,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,2019,01,005000,117,0025,00820,00000,30352.0,142,13820,NaN,NaN,35007,3,6,12,Alabaster,47,47,15849853,0,0,0,0,0,0,0,0,0,47,47,15849853,0,0,0,0,0,0,0,0,0
4,2019,01,006000,095,0030,00988,00000,21160.0,999,99999,NaN,NaN,35950,3,6,12,Albertville,29,29,5553000,0,0,0,0,0,0,1,5,264000,29,29,5553000,0,0,0,0,0,0,1,5,264000
5,2019,01,007000,123,0035,01132,00000,14875.0,999,99999,NaN,NaN,350110552,3,6,12,Alexander City,17,17,3637837,0,0,0,0,0,0,0,0,0,17,17,3637837,0,0,0,0,0,0,0,0,0


In [55]:
vertical_concatso = pd.concat([dfso1,dfso2, dfso3,dfso4,dfso5], axis=0)

In [56]:
dfso=vertical_concatso

In [57]:
dfso

,Survey Date,State Code,6-Digit ID,County Code,Census Place Code,FIPS Place Code,FIPS MCD Code,Pop,CSA Code,CBSA Code,Footnote Code,Central City,Zip Code,Region Code,Division Code,Number of Months Rep,Place Name,Place Bldgs,1-unit Units,1-unit Value,2-units Bldgs,2-units Units,2-units Value,3-4 units Bldgs,3-4 units Units,3-4 units Value,5+ units Bldgs,5+ units Units,5+ units Value,1-unit rep Bldgs,1-unit rep Units,1-unit rep Value,2-units rep Bldgs,2-units rep Units,2-units rep Value,3-4 units rep Bldgs,3-4 units rep Units,3-4 units rep Value,5+ units rep Bldgs,5+ units rep Units,5+ units rep Value
1,2015,01,001000,067,0020,00124,00000,2688.0,222,20020,NaN,NaN,36310,3,6,12,Abbeville,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2015,01,002000,073,0030,00460,00000,4522.0,142,13820,NaN,NaN,35005,3,6,12,Adamsville,1,1,200596,0,0,0,0,0,0,0,0,0,1,1,200596,0,0,0,0,0,0,0,0,0
3,2015,01,005000,117,0025,00820,00000,30352.0,142,13820,NaN,NaN,35007,3,6,12,Alabaster,109,109,33075811,0,0,0,0,0,0,0,0,0,109,109,33075811,0,0,0,0,0,0,0,0,0
4,2015,01,006000,095,0030,00988,00000,21160.0,999,99999,NaN,NaN,35950,3,6,12,Albertville,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,2015,01,007000,123,0035,01132,00000,14875.0,999,99999,NaN,NaN,350110552,3,6,12,Alexander City,18,18,4124433,0,0,0,0,0,0,0,0,0,18,18,4124433,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4461,2019,54,577000,079,1730,87988,00000,2301.0,170,26580,NaN,NaN,252130596,3,5,12,Winfield town,3,3,495000,0,0,0,0,0,0,0,0,0,3,3,495000,0,0,0,0,0,0,0,0,0
4462,2019,54,579000,105,NaN,00000,00000,4894.0,425,37620,NaN,NaN,26143,3,5,0,Wirt County,9,9,1368000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4463,2019,54,583000,107,NaN,99990,00000,40975.0,425,37620,NaN,NaN,26101,3,5,12,Wood County Unincorporated Area,43,43,9822556,1,2,90000,12,48,3840000,0,0,0,43,43,9822556,1,2,90000,12,48,3840000,0,0,0
4464,2019,54,585000,049,1745,88708,00000,158.0,999,99999,NaN,NaN,26591,3,5,0,Worthington town,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [58]:
dfso=dfso.reset_index()

In [59]:
# Drop a specific column, e.g., 'Place Name'
dfso = dfso.drop(columns=['index'])

In [60]:
dfso

,Survey Date,State Code,6-Digit ID,County Code,Census Place Code,FIPS Place Code,FIPS MCD Code,Pop,CSA Code,CBSA Code,Footnote Code,Central City,Zip Code,Region Code,Division Code,Number of Months Rep,Place Name,Place Bldgs,1-unit Units,1-unit Value,2-units Bldgs,2-units Units,2-units Value,3-4 units Bldgs,3-4 units Units,3-4 units Value,5+ units Bldgs,5+ units Units,5+ units Value,1-unit rep Bldgs,1-unit rep Units,1-unit rep Value,2-units rep Bldgs,2-units rep Units,2-units rep Value,3-4 units rep Bldgs,3-4 units rep Units,3-4 units rep Value,5+ units rep Bldgs,5+ units rep Units,5+ units rep Value
0,2015,01,001000,067,0020,00124,00000,2688.0,222,20020,NaN,NaN,36310,3,6,12,Abbeville,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2015,01,002000,073,0030,00460,00000,4522.0,142,13820,NaN,NaN,35005,3,6,12,Adamsville,1,1,200596,0,0,0,0,0,0,0,0,0,1,1,200596,0,0,0,0,0,0,0,0,0
2,2015,01,005000,117,0025,00820,00000,30352.0,142,13820,NaN,NaN,35007,3,6,12,Alabaster,109,109,33075811,0,0,0,0,0,0,0,0,0,109,109,33075811,0,0,0,0,0,0,0,0,0
3,2015,01,006000,095,0030,00988,00000,21160.0,999,99999,NaN,NaN,35950,3,6,12,Albertville,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,2015,01,007000,123,0035,01132,00000,14875.0,999,99999,NaN,NaN,350110552,3,6,12,Alexander City,18,18,4124433,0,0,0,0,0,0,0,0,0,18,18,4124433,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22336,2019,54,577000,079,1730,87988,00000,2301.0,170,26580,NaN,NaN,252130596,3,5,12,Winfield town,3,3,495000,0,0,0,0,0,0,0,0,0,3,3,495000,0,0,0,0,0,0,0,0,0
22337,2019,54,579000,105,NaN,00000,00000,4894.0,425,37620,NaN,NaN,26143,3,5,0,Wirt County,9,9,1368000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
22338,2019,54,583000,107,NaN,99990,00000,40975.0,425,37620,NaN,NaN,26101,3,5,12,Wood County Unincorporated Area,43,43,9822556,1,2,90000,12,48,3840000,0,0,0,43,43,9822556,1,2,90000,12,48,3840000,0,0,0
22339,2019,54,585000,049,1745,88708,00000,158.0,999,99999,NaN,NaN,26591,3,5,0,Worthington town,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [61]:
import pandas as pd

# Assuming your data is in a CSV file named 'data.csv'
dataso = dfso

# Select the columns of interest
selected_columnsso = ['FIPS Place Code','Zip Code','Survey Date', '1-unit rep Units', 'Place Name']

# Extract the desired data
extracted_datasso = dataso[selected_columnsso]


In [62]:
#choosing the column with 1-unit rep unit data
extracted_datasso

,FIPS Place Code,Zip Code,Survey Date,1-unit rep Units,Place Name
0,00124,36310,2015,0,Abbeville
1,00460,35005,2015,1,Adamsville
2,00820,35007,2015,109,Alabaster
3,00988,35950,2015,0,Albertville
4,01132,350110552,2015,18,Alexander City
...,...,...,...,...,...
22336,87988,252130596,2019,3,Winfield town
22337,00000,26143,2019,0,Wirt County
22338,99990,26101,2019,43,Wood County Unincorporated Area
22339,88708,26591,2019,0,Worthington town


In [63]:
# Group by 'Zip Code', 'FIPS Place Code', and 'Place Name', and sum '1-unit rep Units'
grouped_df1so = extracted_datasso.groupby(['Zip Code', 'FIPS Place Code', 'Place Name', 'Survey Date'])['1-unit rep Units'].sum().reset_index()

# Pivot the DataFrame to have years as columns
pivot_df1so = grouped_df1so.pivot_table(index=['Zip Code', 'FIPS Place Code', 'Place Name'], columns='Survey Date', values='1-unit rep Units', fill_value=0)

# Reset the index to have 'Zip Code', 'FIPS Place Code', and 'Place Name' as columns
pivot_df1so.reset_index(inplace=True)

# Rename the columns
pivot_df1so.columns = ['Zip Code', 'FIPS Place Code', 'Place Name', '2015', '2016', '2017', '2018', '2019']



# Display the pivot DataFrame
print(pivot_df1so)

         Zip Code FIPS Place Code                             Place Name  \
0     19709                47030                         Middletown town   
1     19711                50670                                  Newark   
2     19720                50800                              New Castle   
3     19720                99990   New Castle County Unincorporated Area   
4     197300111            54050                             Odessa town   
...           ...             ...                                    ...   
4710  79855                75032                           Van Horn town   
4711  79901                24000                                 El Paso   
4712  79927                68636                                 Socorro   
4713  79928                34832                            Horizon City   
4714  983822530            62804                                Rockport   

      2015  2016  2017  2018  2019  
0      128   112   158   188   149  
1       19   

In [64]:
pivot_df1so

,Zip Code,FIPS Place Code,Place Name,2015,2016,2017,2018,2019
0,19709,47030,Middletown town,128,112,158,188,149
1,19711,50670,Newark,19,21,8,20,51
2,19720,50800,New Castle,5,8,1,3,4
3,19720,99990,New Castle County Unincorporated Area,1021,952,928,837,1046
4,197300111,54050,Odessa town,0,0,1,0,0
...,...,...,...,...,...,...,...,...
4710,79855,75032,Van Horn town,1,3,6,7,8
4711,79901,24000,El Paso,2282,2014,2020,1588,1873
4712,79927,68636,Socorro,126,73,173,112,316
4713,79928,34832,Horizon City,68,101,33,45,234


In [65]:
dfso

,Survey Date,State Code,6-Digit ID,County Code,Census Place Code,FIPS Place Code,FIPS MCD Code,Pop,CSA Code,CBSA Code,Footnote Code,Central City,Zip Code,Region Code,Division Code,Number of Months Rep,Place Name,Place Bldgs,1-unit Units,1-unit Value,2-units Bldgs,2-units Units,2-units Value,3-4 units Bldgs,3-4 units Units,3-4 units Value,5+ units Bldgs,5+ units Units,5+ units Value,1-unit rep Bldgs,1-unit rep Units,1-unit rep Value,2-units rep Bldgs,2-units rep Units,2-units rep Value,3-4 units rep Bldgs,3-4 units rep Units,3-4 units rep Value,5+ units rep Bldgs,5+ units rep Units,5+ units rep Value
0,2015,01,001000,067,0020,00124,00000,2688.0,222,20020,NaN,NaN,36310,3,6,12,Abbeville,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2015,01,002000,073,0030,00460,00000,4522.0,142,13820,NaN,NaN,35005,3,6,12,Adamsville,1,1,200596,0,0,0,0,0,0,0,0,0,1,1,200596,0,0,0,0,0,0,0,0,0
2,2015,01,005000,117,0025,00820,00000,30352.0,142,13820,NaN,NaN,35007,3,6,12,Alabaster,109,109,33075811,0,0,0,0,0,0,0,0,0,109,109,33075811,0,0,0,0,0,0,0,0,0
3,2015,01,006000,095,0030,00988,00000,21160.0,999,99999,NaN,NaN,35950,3,6,12,Albertville,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,2015,01,007000,123,0035,01132,00000,14875.0,999,99999,NaN,NaN,350110552,3,6,12,Alexander City,18,18,4124433,0,0,0,0,0,0,0,0,0,18,18,4124433,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22336,2019,54,577000,079,1730,87988,00000,2301.0,170,26580,NaN,NaN,252130596,3,5,12,Winfield town,3,3,495000,0,0,0,0,0,0,0,0,0,3,3,495000,0,0,0,0,0,0,0,0,0
22337,2019,54,579000,105,NaN,00000,00000,4894.0,425,37620,NaN,NaN,26143,3,5,0,Wirt County,9,9,1368000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
22338,2019,54,583000,107,NaN,99990,00000,40975.0,425,37620,NaN,NaN,26101,3,5,12,Wood County Unincorporated Area,43,43,9822556,1,2,90000,12,48,3840000,0,0,0,43,43,9822556,1,2,90000,12,48,3840000,0,0,0
22339,2019,54,585000,049,1745,88708,00000,158.0,999,99999,NaN,NaN,26591,3,5,0,Worthington town,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [66]:
import pandas as pd

# Assuming your data is in a CSV file named 'data.csv'
dataso2 = dfso

# Select the columns of interest
selected_columnsso2 = ['FIPS Place Code','Zip Code','Survey Date', '2-units rep Units','3-4 units rep Units','5+ units rep Units', 'Place Name']

# Extract the desired data
extracted_datamso2 = dataso2[selected_columnsso2]

In [67]:
extracted_datamso2

,FIPS Place Code,Zip Code,Survey Date,2-units rep Units,3-4 units rep Units,5+ units rep Units,Place Name
0,00124,36310,2015,0,0,0,Abbeville
1,00460,35005,2015,0,0,0,Adamsville
2,00820,35007,2015,0,0,0,Alabaster
3,00988,35950,2015,0,0,0,Albertville
4,01132,350110552,2015,0,0,0,Alexander City
...,...,...,...,...,...,...,...
22336,87988,252130596,2019,0,0,0,Winfield town
22337,00000,26143,2019,0,0,0,Wirt County
22338,99990,26101,2019,2,48,0,Wood County Unincorporated Area
22339,88708,26591,2019,0,0,0,Worthington town


In [68]:
# Group by 'Zip Code,' 'FIPS Place Code,' 'Place Name,' and 'Survey Date,' and sum the units columns
grouped_dfmso = extracted_datamso2.groupby(['Zip Code', 'FIPS Place Code', 'Place Name', 'Survey Date']).agg({
    '2-units rep Units': 'sum',
    '3-4 units rep Units': 'sum',
    '5+ units rep Units': 'sum'
}).reset_index()

# Create a new column 'multi unit' by summing the '2-units rep Units,' '3-4 units rep Units,' and '5+ units rep Units'
grouped_dfmso['multi unit'] = grouped_dfmso['2-units rep Units'] + grouped_dfmso['3-4 units rep Units'] + grouped_dfmso['5+ units rep Units']



# Display the grouped DataFrame
print(grouped_dfmso)





          Zip Code FIPS Place Code       Place Name Survey Date  \
0      19709                47030   Middletown town        2015   
1      19709                47030   Middletown town        2016   
2      19709                47030   Middletown town        2017   
3      19709                47030   Middletown town        2018   
4      19709                47030   Middletown town        2019   
...            ...             ...              ...         ...   
22333  79928                34832      Horizon City        2016   
22334  79928                34832      Horizon City        2017   
22335  79928                34832      Horizon City        2018   
22336  79928                34832      Horizon City        2019   
22337  983822530            62804          Rockport        2015   

      2-units rep Units 3-4 units rep Units 5+ units rep Units multi unit  
0                     4                   0                 96       4096  
1                     6                   0

In [69]:
grouped_dfmso

,Zip Code,FIPS Place Code,Place Name,Survey Date,2-units rep Units,3-4 units rep Units,5+ units rep Units,multi unit
0,19709,47030,Middletown town,2015,4,0,96,4096
1,19709,47030,Middletown town,2016,6,0,120,60120
2,19709,47030,Middletown town,2017,4,0,144,40144
3,19709,47030,Middletown town,2018,2,0,28,2028
4,19709,47030,Middletown town,2019,6,0,168,60168
...,...,...,...,...,...,...,...,...
22333,79928,34832,Horizon City,2016,0,0,0,000
22334,79928,34832,Horizon City,2017,0,0,0,000
22335,79928,34832,Horizon City,2018,0,0,0,000
22336,79928,34832,Horizon City,2019,0,0,0,000


In [70]:
# Pivot the DataFrame to have years as columns
pivot_dfmso = grouped_dfmso.pivot_table(index=['Zip Code', 'FIPS Place Code', 'Place Name'], columns='Survey Date', values='multi unit', fill_value=0)

# Reset the index to have 'Zip Code', 'FIPS Place Code', and 'Place Name' as columns
pivot_dfmso.reset_index(inplace=True)

# Rename the columns
pivot_dfmso.columns = ['Zip Code', 'FIPS Place Code', 'Place Name', '2015', '2016', '2017', '2018', '2019']



# Display the pivot DataFrame
print(pivot_dfmso)

         Zip Code FIPS Place Code                             Place Name  \
0     19709                47030                         Middletown town   
1     19711                50670                                  Newark   
2     19720                50800                              New Castle   
3     19720                99990   New Castle County Unincorporated Area   
4     197300111            54050                             Odessa town   
...           ...             ...                                    ...   
4710  79855                75032                           Van Horn town   
4711  79901                24000                                 El Paso   
4712  79927                68636                                 Socorro   
4713  79928                34832                            Horizon City   
4714  983822530            62804                                Rockport   

          2015     2016      2017      2018     2019  
0         4096    60120     4014

In [71]:
pivot_dfmso

,Zip Code,FIPS Place Code,Place Name,2015,2016,2017,2018,2019
0,19709,47030,Middletown town,4096,60120,40144,2028,60168
1,19711,50670,Newark,0,224,320,0,0
2,19720,50800,New Castle,0,0,120,0,0
3,19720,99990,New Castle County Unincorporated Area,269,120,138,0,60
4,197300111,54050,Odessa town,0,0,0,0,0
...,...,...,...,...,...,...,...,...
4710,79855,75032,Van Horn town,0,0,0,0,0
4711,79901,24000,El Paso,64361869,8471674,46187664,11822481,4824341
4712,79927,68636,Socorro,40,600,5,3680,161560
4713,79928,34832,Horizon City,0,0,0,0,0


# **Repeating all steps for West Region**

In [72]:
dfwo1 = pd.read_table("https://www2.census.gov/econ/bps/Place/West%20Region/we2015a.txt", sep=",",
                  skipinitialspace=True) #read the online table

# the web data table has some format issue, this block of code is to address that issue
dfwo1["Unnamed 41"] = np.nan
dfwo1 = dfwo1.shift(1, axis=1)
dfwo1.Survey = dfwo1.index
dfwo1.index = np.arange(len(dfwo1))

dict_names = {}
second  = dfwo1.iloc[0].fillna("")
for i in range(len(dfwo1.columns)):
    first = dfwo1.columns[i]
    if first.startswith("Unnamed"):
        if dfwo1.columns[i-1].startswith("Unnamed"):
            first = dfwo1.columns[i+1]
        else:
            first = dfwo1.columns[i-1]
    #         new = first+second[i]
    # else:
    new = first+" "+second[i]
    dict_names[dfwo1.columns[i]]= new
dfwo1 = dfwo1.rename(columns = dict_names).drop([0])

# show the first five records
dfwo1.head()

,Survey Date,State Code,6-Digit ID,County Code,Census Place Code,FIPS Place Code,FIPS MCD Code,Pop,CSA Code,CBSA Code,Footnote Code,Central City,Zip Code,Region Code,Division Code,Number of Months Rep,Place Name,Place Bldgs,1-unit Units,1-unit Value,2-units Bldgs,2-units Units,2-units Value,3-4 units Bldgs,3-4 units Units,3-4 units Value,5+ units Bldgs,5+ units Units,5+ units Value,1-unit rep Bldgs,1-unit rep Units,1-unit rep Value,2-units rep Bldgs,2-units rep Units,2-units rep Value,3-4 units rep Bldgs,3-4 units rep Units,3-4 units rep Value,5+ units rep Bldgs,5+ units rep Units,5+ units rep Value
1,2015,02,009000,013,0070,01090,00000,1027.0,999,99999,NaN,NaN,99503,4,9,0,Akutan,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2015,02,041000,020,0140,03000,00000,291826.0,999,11260,NaN,1,995196650,4,9,0,Municipality of Anchorage,648,648,202797264,1,2,322399,12,36,6272178,17,148,23398143,0,0,0,0,0,0,0,0,0,0,0,0
3,2015,02,049000,105,0160,03440,00000,459.0,999,99999,NaN,NaN,998200189,4,9,0,Angoon,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,2015,02,085000,050,0310,06520,00000,6080.0,999,99999,NaN,NaN,99559,4,9,12,Bethel,6,6,1300000,2,4,700000,0,0,0,0,0,0,6,6,1300000,2,4,700000,0,0,0,0,0,0
5,2015,02,097000,060,NaN,99990,00000,997.0,999,99999,NaN,NaN,996330189,4,9,12,Bristol Bay Borough,3,3,401500,0,0,0,0,0,0,0,0,0,3,3,401500,0,0,0,0,0,0,0,0,0


In [73]:
dfwo2 = pd.read_table("https://www2.census.gov/econ/bps/Place/West%20Region/we2016a.txt", sep=",",
                  skipinitialspace=True) #read the online table

# the web data table has some format issue, this block of code is to address that issue
dfwo2["Unnamed 41"] = np.nan
dfwo2 = dfwo2.shift(1, axis=1)
dfwo2.Survey = dfwo2.index
dfwo2.index = np.arange(len(dfwo2))

dict_names = {}
second  = dfwo2.iloc[0].fillna("")
for i in range(len(dfwo2.columns)):
    first = dfwo2.columns[i]
    if first.startswith("Unnamed"):
        if dfwo2.columns[i-1].startswith("Unnamed"):
            first = dfwo2.columns[i+1]
        else:
            first = dfwo2.columns[i-1]
    #         new = first+second[i]
    # else:
    new = first+" "+second[i]
    dict_names[dfwo2.columns[i]]= new
dfwo2 = dfwo2.rename(columns = dict_names).drop([0])

# show the first five records
dfwo2.head()

,Survey Date,State Code,6-Digit ID,County Code,Census Place Code,FIPS Place Code,FIPS MCD Code,Pop,CSA Code,CBSA Code,Footnote Code,Central City,Zip Code,Region Code,Division Code,Number of Months Rep,Place Name,Place Bldgs,1-unit Units,1-unit Value,2-units Bldgs,2-units Units,2-units Value,3-4 units Bldgs,3-4 units Units,3-4 units Value,5+ units Bldgs,5+ units Units,5+ units Value,1-unit rep Bldgs,1-unit rep Units,1-unit rep Value,2-units rep Bldgs,2-units rep Units,2-units rep Value,3-4 units rep Bldgs,3-4 units rep Units,3-4 units rep Value,5+ units rep Bldgs,5+ units rep Units,5+ units rep Value
1,2016,02,009000,013,0070,01090,00000,1027.0,999,99999,NaN,NaN,99503,4,9,1,Akutan,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2016,02,041000,020,0140,03000,00000,291826.0,999,11260,NaN,1,995196650,4,9,0,Municipality of Anchorage,719,719,225284620,0,0,0,12,37,6422957,18,174,27508629,0,0,0,0,0,0,0,0,0,0,0,0
3,2016,02,049000,105,0160,03440,00000,459.0,999,99999,2,NaN,998200189,4,9,12,Angoon,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,2016,02,085000,050,0310,06520,00000,6080.0,999,99999,NaN,NaN,99559,4,9,12,Bethel,6,6,1800000,1,2,450000,0,0,0,0,0,0,6,6,1800000,1,2,450000,0,0,0,0,0,0
5,2016,02,097000,060,NaN,99990,00000,997.0,999,99999,NaN,NaN,996330189,4,9,12,Bristol Bay Borough,2,2,450000,0,0,0,0,0,0,0,0,0,2,2,450000,0,0,0,0,0,0,0,0,0


In [74]:
dfwo3 = pd.read_table("https://www2.census.gov/econ/bps/Place/West%20Region/we2017a.txt", sep=",",
                  skipinitialspace=True) #read the online table

# the web data table has some format issue, this block of code is to address that issue
dfwo3["Unnamed 41"] = np.nan
dfwo3 = dfwo3.shift(1, axis=1)
dfwo3.Survey = dfwo3.index
dfwo3.index = np.arange(len(dfwo3))

dict_names = {}
second  = dfwo3.iloc[0].fillna("")
for i in range(len(dfwo3.columns)):
    first = dfwo3.columns[i]
    if first.startswith("Unnamed"):
        if dfwo3.columns[i-1].startswith("Unnamed"):
            first = dfwo3.columns[i+1]
        else:
            first = dfwo3.columns[i-1]
    #         new = first+second[i]
    # else:
    new = first+" "+second[i]
    dict_names[dfwo3.columns[i]]= new
dfwo3 = dfwo3.rename(columns = dict_names).drop([0])

# show the first five records
dfwo3.head()

,Survey Date,State Code,6-Digit ID,County Code,Census Place Code,FIPS Place Code,FIPS MCD Code,Pop,CSA Code,CBSA Code,Footnote Code,Central City,Zip Code,Region Code,Division Code,Number of Months Rep,Place Name,Place Bldgs,1-unit Units,1-unit Value,2-units Bldgs,2-units Units,2-units Value,3-4 units Bldgs,3-4 units Units,3-4 units Value,5+ units Bldgs,5+ units Units,5+ units Value,1-unit rep Bldgs,1-unit rep Units,1-unit rep Value,2-units rep Bldgs,2-units rep Units,2-units rep Value,3-4 units rep Bldgs,3-4 units rep Units,3-4 units rep Value,5+ units rep Bldgs,5+ units rep Units,5+ units rep Value
1,2017,02,009000,013,0070,01090,00000,1027.0,999,99999,NaN,NaN,99503,4,9,12,Akutan,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2017,02,041000,020,0140,03000,00000,291826.0,999,11260,NaN,1,995196650,4,9,0,Municipality of Anchorage,800,800,250599715,0,0,0,14,47,8165231,19,172,27192436,0,0,0,0,0,0,0,0,0,0,0,0
3,2017,02,049000,105,0160,03440,00000,459.0,999,99999,2,NaN,998200189,4,9,0,Angoon,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,2017,02,085000,050,0310,06520,00000,6080.0,999,99999,NaN,NaN,99559,4,9,12,Bethel,15,15,3522104,0,0,0,0,0,0,0,0,0,15,15,3522104,0,0,0,0,0,0,0,0,0
5,2017,02,097000,060,NaN,99990,00000,997.0,999,99999,NaN,NaN,996330189,4,9,11,Bristol Bay Borough,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [75]:
dfwo4 = pd.read_table("https://www2.census.gov/econ/bps/Place/West%20Region/we2018a.txt", sep=",",
                  skipinitialspace=True) #read the online table

# the web data table has some format issue, this block of code is to address that issue
dfwo4["Unnamed 41"] = np.nan
dfwo4 = dfwo4.shift(1, axis=1)
dfwo4.Survey = dfwo4.index
dfwo4.index = np.arange(len(dfwo4))

dict_names = {}
second  = dfwo4.iloc[0].fillna("")
for i in range(len(dfwo4.columns)):
    first = dfwo4.columns[i]
    if first.startswith("Unnamed"):
        if dfwo4.columns[i-1].startswith("Unnamed"):
            first = dfwo4.columns[i+1]
        else:
            first = dfwo4.columns[i-1]
    #         new = first+second[i]
    # else:
    new = first+" "+second[i]
    dict_names[dfwo4.columns[i]]= new
dfwo4 = dfwo4.rename(columns = dict_names).drop([0])

# show the first five records
dfwo4.head()

,Survey Date,State Code,6-Digit ID,County Code,Census Place Code,FIPS Place Code,FIPS MCD Code,Pop,CSA Code,CBSA Code,Footnote Code,Central City,Zip Code,Region Code,Division Code,Number of Months Rep,Place Name,Place Bldgs,1-unit Units,1-unit Value,2-units Bldgs,2-units Units,2-units Value,3-4 units Bldgs,3-4 units Units,3-4 units Value,5+ units Bldgs,5+ units Units,5+ units Value,1-unit rep Bldgs,1-unit rep Units,1-unit rep Value,2-units rep Bldgs,2-units rep Units,2-units rep Value,3-4 units rep Bldgs,3-4 units rep Units,3-4 units rep Value,5+ units rep Bldgs,5+ units rep Units,5+ units rep Value
1,2018,02,009000,013,0070,01090,00000,1027.0,999,99999,NaN,NaN,99503,4,9,12,Akutan,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2018,02,041000,020,0140,03000,00000,291826.0,999,11260,NaN,1,995196650,4,9,0,Municipality of Anchorage,869,869,272231334,0,0,0,13,48,8337474,19,166,26243863,0,0,0,0,0,0,0,0,0,0,0,0
3,2018,02,049000,105,0160,03440,00000,459.0,999,99999,2,NaN,998200189,4,9,0,Angoon,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,2018,02,085000,050,0310,06520,00000,6080.0,999,99999,NaN,NaN,99559,4,9,12,Bethel,1,1,269793,0,0,0,0,0,0,0,0,0,1,1,269793,0,0,0,0,0,0,0,0,0
5,2018,02,097000,060,NaN,99990,00000,997.0,999,99999,NaN,NaN,996330189,4,9,10,Bristol Bay Borough,2,2,310000,0,0,0,0,0,0,0,0,0,2,2,310000,0,0,0,0,0,0,0,0,0


In [76]:
dfwo5 = pd.read_table("https://www2.census.gov/econ/bps/Place/West%20Region/we2019a.txt", sep=",",
                  skipinitialspace=True) #read the online table

# the web data table has some format issue, this block of code is to address that issue
dfwo5["Unnamed 41"] = np.nan
dfwo5 = dfwo5.shift(1, axis=1)
dfwo5.Survey = dfwo5.index
dfwo5.index = np.arange(len(dfwo5))

dict_names = {}
second  = dfwo5.iloc[0].fillna("")
for i in range(len(dfwo5.columns)):
    first = dfwo5.columns[i]
    if first.startswith("Unnamed"):
        if dfwo5.columns[i-1].startswith("Unnamed"):
            first = dfwo5.columns[i+1]
        else:
            first = dfwo5.columns[i-1]
    #         new = first+second[i]
    # else:
    new = first+" "+second[i]
    dict_names[dfwo5.columns[i]]= new
dfwo5 = dfwo5.rename(columns = dict_names).drop([0])

# show the first five records
dfwo5.head()

,Survey Date,State Code,6-Digit ID,County Code,Census Place Code,FIPS Place Code,FIPS MCD Code,Pop,CSA Code,CBSA Code,Footnote Code,Central City,Zip Code,Region Code,Division Code,Number of Months Rep,Place Name,Place Bldgs,1-unit Units,1-unit Value,2-units Bldgs,2-units Units,2-units Value,3-4 units Bldgs,3-4 units Units,3-4 units Value,5+ units Bldgs,5+ units Units,5+ units Value,1-unit rep Bldgs,1-unit rep Units,1-unit rep Value,2-units rep Bldgs,2-units rep Units,2-units rep Value,3-4 units rep Bldgs,3-4 units rep Units,3-4 units rep Value,5+ units rep Bldgs,5+ units rep Units,5+ units rep Value
1,2019,02,009000,013,0070,01090,00000,1027.0,999,99999,NaN,NaN,99503,4,9,12,Akutan,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2019,02,041000,020,0140,03000,00000,291826.0,999,11260,NaN,1,995196650,4,9,0,Municipality of Anchorage,838,838,262513354,0,0,0,13,48,8338126,20,173,27350531,0,0,0,0,0,0,0,0,0,0,0,0
3,2019,02,049000,105,0160,03440,00000,459.0,999,99999,2,NaN,998200189,4,9,0,Angoon,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,2019,02,085000,050,0310,06520,00000,6080.0,999,99999,NaN,NaN,99559,4,9,12,Bethel,8,8,2267366,3,6,1037349,1,3,372759,0,0,0,8,8,2267366,3,6,1037349,1,3,372759,0,0,0
5,2019,02,097000,060,NaN,99990,00000,997.0,999,99999,NaN,NaN,996330189,4,9,12,Bristol Bay Borough,8,8,925000,0,0,0,0,0,0,0,0,0,8,8,925000,0,0,0,0,0,0,0,0,0


In [77]:
vertical_concatwo = pd.concat([dfwo1,dfwo2, dfwo3,dfwo4,dfwo5], axis=0)

In [78]:
dfwo=vertical_concatwo

In [79]:
dfwo

,Survey Date,State Code,6-Digit ID,County Code,Census Place Code,FIPS Place Code,FIPS MCD Code,Pop,CSA Code,CBSA Code,Footnote Code,Central City,Zip Code,Region Code,Division Code,Number of Months Rep,Place Name,Place Bldgs,1-unit Units,1-unit Value,2-units Bldgs,2-units Units,2-units Value,3-4 units Bldgs,3-4 units Units,3-4 units Value,5+ units Bldgs,5+ units Units,5+ units Value,1-unit rep Bldgs,1-unit rep Units,1-unit rep Value,2-units rep Bldgs,2-units rep Units,2-units rep Value,3-4 units rep Bldgs,3-4 units rep Units,3-4 units rep Value,5+ units rep Bldgs,5+ units rep Units,5+ units rep Value
1,2015,02,009000,013,0070,01090,00000,1027.0,999,99999,NaN,NaN,99503,4,9,0,Akutan,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2015,02,041000,020,0140,03000,00000,291826.0,999,11260,NaN,1,995196650,4,9,0,Municipality of Anchorage,648,648,202797264,1,2,322399,12,36,6272178,17,148,23398143,0,0,0,0,0,0,0,0,0,0,0,0
3,2015,02,049000,105,0160,03440,00000,459.0,999,99999,NaN,NaN,998200189,4,9,0,Angoon,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,2015,02,085000,050,0310,06520,00000,6080.0,999,99999,NaN,NaN,99559,4,9,12,Bethel,6,6,1300000,2,4,700000,0,0,0,0,0,0,6,6,1300000,2,4,700000,0,0,0,0,0,0
5,2015,02,097000,060,NaN,99990,00000,997.0,999,99999,NaN,NaN,996330189,4,9,12,Bristol Bay Borough,3,3,401500,0,0,0,0,0,0,0,0,0,3,3,401500,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2043,2019,56,457000,037,0465,81300,00000,451.0,999,99999,NaN,NaN,82336,4,8,12,Wamsutter town,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2044,2019,56,469000,031,0478,83040,00000,3627.0,999,99999,NaN,NaN,82201,4,8,0,Wheatland town,2,2,382500,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2045,2019,56,473000,043,0487,84925,00000,5487.0,999,99999,NaN,NaN,82401,4,8,12,Worland,1,1,275000,0,0,0,0,0,0,0,0,0,1,1,275000,0,0,0,0,0,0,0,0,0
2046,2019,56,477000,005,0490,85015,00000,1807.0,999,99999,NaN,NaN,827320070,4,8,12,Wright town,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [80]:
dfwo=dfwo.reset_index()

In [81]:
# Drop a specific column, e.g., 'Place Name'
dfwo = dfwo.drop(columns=['index'])

In [82]:
dfwo

,Survey Date,State Code,6-Digit ID,County Code,Census Place Code,FIPS Place Code,FIPS MCD Code,Pop,CSA Code,CBSA Code,Footnote Code,Central City,Zip Code,Region Code,Division Code,Number of Months Rep,Place Name,Place Bldgs,1-unit Units,1-unit Value,2-units Bldgs,2-units Units,2-units Value,3-4 units Bldgs,3-4 units Units,3-4 units Value,5+ units Bldgs,5+ units Units,5+ units Value,1-unit rep Bldgs,1-unit rep Units,1-unit rep Value,2-units rep Bldgs,2-units rep Units,2-units rep Value,3-4 units rep Bldgs,3-4 units rep Units,3-4 units rep Value,5+ units rep Bldgs,5+ units rep Units,5+ units rep Value
0,2015,02,009000,013,0070,01090,00000,1027.0,999,99999,NaN,NaN,99503,4,9,0,Akutan,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2015,02,041000,020,0140,03000,00000,291826.0,999,11260,NaN,1,995196650,4,9,0,Municipality of Anchorage,648,648,202797264,1,2,322399,12,36,6272178,17,148,23398143,0,0,0,0,0,0,0,0,0,0,0,0
2,2015,02,049000,105,0160,03440,00000,459.0,999,99999,NaN,NaN,998200189,4,9,0,Angoon,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,2015,02,085000,050,0310,06520,00000,6080.0,999,99999,NaN,NaN,99559,4,9,12,Bethel,6,6,1300000,2,4,700000,0,0,0,0,0,0,6,6,1300000,2,4,700000,0,0,0,0,0,0
4,2015,02,097000,060,NaN,99990,00000,997.0,999,99999,NaN,NaN,996330189,4,9,12,Bristol Bay Borough,3,3,401500,0,0,0,0,0,0,0,0,0,3,3,401500,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10288,2019,56,457000,037,0465,81300,00000,451.0,999,99999,NaN,NaN,82336,4,8,12,Wamsutter town,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
10289,2019,56,469000,031,0478,83040,00000,3627.0,999,99999,NaN,NaN,82201,4,8,0,Wheatland town,2,2,382500,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
10290,2019,56,473000,043,0487,84925,00000,5487.0,999,99999,NaN,NaN,82401,4,8,12,Worland,1,1,275000,0,0,0,0,0,0,0,0,0,1,1,275000,0,0,0,0,0,0,0,0,0
10291,2019,56,477000,005,0490,85015,00000,1807.0,999,99999,NaN,NaN,827320070,4,8,12,Wright town,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [83]:
import pandas as pd

# Assuming your data is in a CSV file named 'data.csv'
datawo = dfwo

# Select the columns of interest
selected_columnswo = ['FIPS Place Code','Zip Code','Survey Date', '1-unit rep Units', 'Place Name']

# Extract the desired data
extracted_dataswo = dataso[selected_columnswo]


In [84]:
#choosing the column with 1-unit rep unit data
extracted_dataswo

,FIPS Place Code,Zip Code,Survey Date,1-unit rep Units,Place Name
0,00124,36310,2015,0,Abbeville
1,00460,35005,2015,1,Adamsville
2,00820,35007,2015,109,Alabaster
3,00988,35950,2015,0,Albertville
4,01132,350110552,2015,18,Alexander City
...,...,...,...,...,...
22336,87988,252130596,2019,3,Winfield town
22337,00000,26143,2019,0,Wirt County
22338,99990,26101,2019,43,Wood County Unincorporated Area
22339,88708,26591,2019,0,Worthington town


In [85]:
# Group by 'Zip Code', 'FIPS Place Code', and 'Place Name', and sum '1-unit rep Units'
grouped_df1wo = extracted_dataswo.groupby(['Zip Code', 'FIPS Place Code', 'Place Name', 'Survey Date'])['1-unit rep Units'].sum().reset_index()

# Pivot the DataFrame to have years as columns
pivot_df1wo = grouped_df1wo.pivot_table(index=['Zip Code', 'FIPS Place Code', 'Place Name'], columns='Survey Date', values='1-unit rep Units', fill_value=0)

# Reset the index to have 'Zip Code', 'FIPS Place Code', and 'Place Name' as columns
pivot_df1wo.reset_index(inplace=True)

# Rename the columns
pivot_df1wo.columns = ['Zip Code', 'FIPS Place Code', 'Place Name', '2015', '2016', '2017', '2018', '2019']



# Display the pivot DataFrame
print(pivot_df1wo)

         Zip Code FIPS Place Code                             Place Name  \
0     19709                47030                         Middletown town   
1     19711                50670                                  Newark   
2     19720                50800                              New Castle   
3     19720                99990   New Castle County Unincorporated Area   
4     197300111            54050                             Odessa town   
...           ...             ...                                    ...   
4710  79855                75032                           Van Horn town   
4711  79901                24000                                 El Paso   
4712  79927                68636                                 Socorro   
4713  79928                34832                            Horizon City   
4714  983822530            62804                                Rockport   

      2015  2016  2017  2018  2019  
0      128   112   158   188   149  
1       19   

In [86]:
pivot_df1wo

,Zip Code,FIPS Place Code,Place Name,2015,2016,2017,2018,2019
0,19709,47030,Middletown town,128,112,158,188,149
1,19711,50670,Newark,19,21,8,20,51
2,19720,50800,New Castle,5,8,1,3,4
3,19720,99990,New Castle County Unincorporated Area,1021,952,928,837,1046
4,197300111,54050,Odessa town,0,0,1,0,0
...,...,...,...,...,...,...,...,...
4710,79855,75032,Van Horn town,1,3,6,7,8
4711,79901,24000,El Paso,2282,2014,2020,1588,1873
4712,79927,68636,Socorro,126,73,173,112,316
4713,79928,34832,Horizon City,68,101,33,45,234


In [87]:
dfwo

,Survey Date,State Code,6-Digit ID,County Code,Census Place Code,FIPS Place Code,FIPS MCD Code,Pop,CSA Code,CBSA Code,Footnote Code,Central City,Zip Code,Region Code,Division Code,Number of Months Rep,Place Name,Place Bldgs,1-unit Units,1-unit Value,2-units Bldgs,2-units Units,2-units Value,3-4 units Bldgs,3-4 units Units,3-4 units Value,5+ units Bldgs,5+ units Units,5+ units Value,1-unit rep Bldgs,1-unit rep Units,1-unit rep Value,2-units rep Bldgs,2-units rep Units,2-units rep Value,3-4 units rep Bldgs,3-4 units rep Units,3-4 units rep Value,5+ units rep Bldgs,5+ units rep Units,5+ units rep Value
0,2015,02,009000,013,0070,01090,00000,1027.0,999,99999,NaN,NaN,99503,4,9,0,Akutan,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2015,02,041000,020,0140,03000,00000,291826.0,999,11260,NaN,1,995196650,4,9,0,Municipality of Anchorage,648,648,202797264,1,2,322399,12,36,6272178,17,148,23398143,0,0,0,0,0,0,0,0,0,0,0,0
2,2015,02,049000,105,0160,03440,00000,459.0,999,99999,NaN,NaN,998200189,4,9,0,Angoon,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,2015,02,085000,050,0310,06520,00000,6080.0,999,99999,NaN,NaN,99559,4,9,12,Bethel,6,6,1300000,2,4,700000,0,0,0,0,0,0,6,6,1300000,2,4,700000,0,0,0,0,0,0
4,2015,02,097000,060,NaN,99990,00000,997.0,999,99999,NaN,NaN,996330189,4,9,12,Bristol Bay Borough,3,3,401500,0,0,0,0,0,0,0,0,0,3,3,401500,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10288,2019,56,457000,037,0465,81300,00000,451.0,999,99999,NaN,NaN,82336,4,8,12,Wamsutter town,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
10289,2019,56,469000,031,0478,83040,00000,3627.0,999,99999,NaN,NaN,82201,4,8,0,Wheatland town,2,2,382500,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
10290,2019,56,473000,043,0487,84925,00000,5487.0,999,99999,NaN,NaN,82401,4,8,12,Worland,1,1,275000,0,0,0,0,0,0,0,0,0,1,1,275000,0,0,0,0,0,0,0,0,0
10291,2019,56,477000,005,0490,85015,00000,1807.0,999,99999,NaN,NaN,827320070,4,8,12,Wright town,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [88]:
import pandas as pd

# Assuming your data is in a CSV file named 'data.csv'
datawo2 = dfwo

# Select the columns of interest
selected_columnswo2 = ['FIPS Place Code','Zip Code','Survey Date', '2-units rep Units','3-4 units rep Units','5+ units rep Units', 'Place Name']

# Extract the desired data
extracted_datamwo2 = datawo2[selected_columnswo2]

In [89]:
extracted_datamwo2

,FIPS Place Code,Zip Code,Survey Date,2-units rep Units,3-4 units rep Units,5+ units rep Units,Place Name
0,01090,99503,2015,0,0,0,Akutan
1,03000,995196650,2015,0,0,0,Municipality of Anchorage
2,03440,998200189,2015,0,0,0,Angoon
3,06520,99559,2015,4,0,0,Bethel
4,99990,996330189,2015,0,0,0,Bristol Bay Borough
...,...,...,...,...,...,...,...
10288,81300,82336,2019,0,0,0,Wamsutter town
10289,83040,82201,2019,0,0,0,Wheatland town
10290,84925,82401,2019,0,0,0,Worland
10291,85015,827320070,2019,0,0,0,Wright town


In [90]:
# Group by 'Zip Code,' 'FIPS Place Code,' 'Place Name,' and 'Survey Date,' and sum the units columns
grouped_dfmwo = extracted_datamwo2.groupby(['Zip Code', 'FIPS Place Code', 'Place Name', 'Survey Date']).agg({
    '2-units rep Units': 'sum',
    '3-4 units rep Units': 'sum',
    '5+ units rep Units': 'sum'
}).reset_index()

# Create a new column 'multi unit' by summing the '2-units rep Units,' '3-4 units rep Units,' and '5+ units rep Units'
grouped_dfmwo['multi unit'] = grouped_dfmwo['2-units rep Units'] + grouped_dfmwo['3-4 units rep Units'] + grouped_dfmwo['5+ units rep Units']



# Display the grouped DataFrame
print(grouped_dfmwo)





          Zip Code FIPS Place Code                 Place Name Survey Date  \
0      59007                04300              Bearcreek town        2015   
1      59007                04300              Bearcreek town        2016   
2      59007                04300              Bearcreek town        2017   
3      59007                04300              Bearcreek town        2018   
4      59007                04300              Bearcreek town        2019   
...            ...             ...                        ...         ...   
10285  99929                86380   Wrangell city and borough        2015   
10286  99929                86380   Wrangell city and borough        2016   
10287  99929                86380   Wrangell city and borough        2017   
10288  99929                86380   Wrangell city and borough        2018   
10289  99929                86380   Wrangell city and borough        2019   

      2-units rep Units 3-4 units rep Units 5+ units rep Units multi unit  

In [91]:
grouped_dfmwo

,Zip Code,FIPS Place Code,Place Name,Survey Date,2-units rep Units,3-4 units rep Units,5+ units rep Units,multi unit
0,59007,04300,Bearcreek town,2015,0,0,0,000
1,59007,04300,Bearcreek town,2016,0,0,0,000
2,59007,04300,Bearcreek town,2017,0,0,0,000
3,59007,04300,Bearcreek town,2018,0,0,0,000
4,59007,04300,Bearcreek town,2019,0,0,0,000
...,...,...,...,...,...,...,...,...
10285,99929,86380,Wrangell city and borough,2015,0,0,0,000
10286,99929,86380,Wrangell city and borough,2016,0,0,0,000
10287,99929,86380,Wrangell city and borough,2017,0,0,0,000
10288,99929,86380,Wrangell city and borough,2018,2,0,0,200


In [92]:
# Pivot the DataFrame to have years as columns
pivot_dfmwo = grouped_dfmwo.pivot_table(index=['Zip Code', 'FIPS Place Code', 'Place Name'], columns='Survey Date', values='multi unit', fill_value=0)

# Reset the index to have 'Zip Code', 'FIPS Place Code', and 'Place Name' as columns
pivot_dfmwo.reset_index(inplace=True)

# Rename the columns
pivot_dfmwo.columns = ['Zip Code', 'FIPS Place Code', 'Place Name', '2015', '2016', '2017', '2018', '2019']



# Display the pivot DataFrame
print(pivot_dfmwo)

         Zip Code FIPS Place Code                 Place Name  2015  2016  \
0     59007                04300              Bearcreek town     0     0   
1     590116850            06475                  Big Timber     0     0   
2     590147789            09475                Bridger town     0     0   
3     59019                16825               Columbus town     0     0   
4     59029                29575               Fromberg town     0     0   
...           ...             ...                        ...   ...   ...   
2143  998400415            70760     Municipality of Skagway   200     0   
2144  998410052            76260             Tenakee Springs     0     0   
2145  999016059            99990   Ketchikan Gateway Borough   240   600   
2146  99921                17740                       Craig   200   230   
2147  99929                86380   Wrangell city and borough     0     0   

      2017  2018  2019  
0        0     0     0  
1        0     0   200  
2        0  

In [93]:
pivot_dfmwo

,Zip Code,FIPS Place Code,Place Name,2015,2016,2017,2018,2019
0,59007,04300,Bearcreek town,0,0,0,0,0
1,590116850,06475,Big Timber,0,0,0,0,200
2,590147789,09475,Bridger town,0,0,0,0,0
3,59019,16825,Columbus town,0,0,0,0,0
4,59029,29575,Fromberg town,0,0,0,0,0
...,...,...,...,...,...,...,...,...
2143,998400415,70760,Municipality of Skagway,200,0,600,6011,1200
2144,998410052,76260,Tenakee Springs,0,0,0,0,0
2145,999016059,99990,Ketchikan Gateway Borough,240,600,800,1200,670
2146,99921,17740,Craig,200,230,1200,200,600


# **Concatenating all regions data**

In [94]:
vertical_concatall = pd.concat([pivot_df, pivot_df1ne, pivot_df1wo ,pivot_df1so], axis=0)

In [95]:
vertical_concatall=vertical_concatall.reset_index()

In [96]:
vertical_concatall

,index,Zip Code,FIPS Place Code,Place Name,2015,2016,2017,2018,2019
0,0,15454,48855,Manilla,0,0,0,1,0
1,1,19372,00000,Tiffin township,0,0,0,3,0
2,2,42112,00000,Milton township,14,11,15,11,12
3,3,43003,00000,Westfield township,0,0,0,0,6
4,4,43011,00000,Hilliar township,9,10,11,10,14
...,...,...,...,...,...,...,...,...,...
23899,4710,79855,75032,Van Horn town,1,3,6,7,8
23900,4711,79901,24000,El Paso,2282,2014,2020,1588,1873
23901,4712,79927,68636,Socorro,126,73,173,112,316
23902,4713,79928,34832,Horizon City,68,101,33,45,234


# **All regions single unit bps for 2014,2015,2016,2017,2018,2019 and csv for it**

In [97]:
# Drop a specific column, e.g., 'Place Name'
vertical_concatall = vertical_concatall.drop(columns=['index'])

In [98]:
# all regions single unit bps for 2014,2015,2016,2017,2018,2019
vertical_concatall

,Zip Code,FIPS Place Code,Place Name,2015,2016,2017,2018,2019
0,15454,48855,Manilla,0,0,0,1,0
1,19372,00000,Tiffin township,0,0,0,3,0
2,42112,00000,Milton township,14,11,15,11,12
3,43003,00000,Westfield township,0,0,0,0,6
4,43011,00000,Hilliar township,9,10,11,10,14
...,...,...,...,...,...,...,...,...
23899,79855,75032,Van Horn town,1,3,6,7,8
23900,79901,24000,El Paso,2282,2014,2020,1588,1873
23901,79927,68636,Socorro,126,73,173,112,316
23902,79928,34832,Horizon City,68,101,33,45,234


# **Building permits for All regions fips place code single unit for 2015,2016,2017,2018,2019 and csv for it**

In [99]:

# Save the pivot DataFrame to a CSV file
vertical_concatall.to_csv('1building_permitssingleunit all_grouped_2015-2019.csv', index=False)


In [100]:
vertical_concatallmu = pd.concat([pivot_dfmwo,pivot_dfmso,pivot_dfmne, pivot_dfm], axis=0)

In [101]:
vertical_concatallmu=vertical_concatallmu.reset_index()

# **Building permits for  All regions fips place code  multi unit  for 2015,2016,2017,2018,2019 and csv for it**

---



In [102]:
vertical_concatallmu

,index,Zip Code,FIPS Place Code,Place Name,2015,2016,2017,2018,2019
0,0,59007,04300,Bearcreek town,0,0,0,0,0
1,1,590116850,06475,Big Timber,0,0,0,0,200
2,2,590147789,09475,Bridger town,0,0,0,0,0
3,3,59019,16825,Columbus town,0,0,0,0,0
4,4,59029,29575,Fromberg town,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...
21332,8494,69358,32830,Morrill village,0,0,0,0,0
21333,8495,69360,42775,Rushville,0,0,0,0,0
21334,8496,693600039,99990,Sheridan County Unincorporated Area,0,0,0,0,0
21335,8497,69361,44245,Scottsbluff,0,12,200,0,0


In [103]:
# Drop a specific column, e.g., 'Place Name'
vertical_concatallmu = vertical_concatallmu.drop(columns=['index'])

In [104]:
vertical_concatallmu

,Zip Code,FIPS Place Code,Place Name,2015,2016,2017,2018,2019
0,59007,04300,Bearcreek town,0,0,0,0,0
1,590116850,06475,Big Timber,0,0,0,0,200
2,590147789,09475,Bridger town,0,0,0,0,0
3,59019,16825,Columbus town,0,0,0,0,0
4,59029,29575,Fromberg town,0,0,0,0,0
...,...,...,...,...,...,...,...,...
21332,69358,32830,Morrill village,0,0,0,0,0
21333,69360,42775,Rushville,0,0,0,0,0
21334,693600039,99990,Sheridan County Unincorporated Area,0,0,0,0,0
21335,69361,44245,Scottsbluff,0,12,200,0,0


In [105]:

# Save the pivot DataFrame to a CSV file
vertical_concatallmu.to_csv('2building_permitsmultipleunit all_grouped_2015-2019.csv', index=False)

# **Gathering county data for all years 2015-2019**

# **County**

In [106]:
#https://www2.census.gov/econ/bps/County/co2014a.txt

In [107]:
dfco1 = pd.read_table("https://www2.census.gov/econ/bps/County/co2015a.txt", sep=",",
                  skipinitialspace=True) #read the online table
                  # the web data table has some format issue, this block of code is to address that issue
dfco1["Unnamed 41"] = np.nan
dfco1 = dfco1.shift(1, axis=1)
dfco1.Survey = dfco1.index
dfco1.index = np.arange(len(dfco1))

dict_names = {}
second  = dfco1.iloc[0].fillna("")
for i in range(len(dfco1.columns)):
    first = dfco1.columns[i]
    if first.startswith("Unnamed"):
        if dfco1.columns[i-1].startswith("Unnamed"):
            first = dfco1.columns[i+1]
        else:
            first = dfco1.columns[i-1]
    #         new = first+second[i]
    # else:
    new = first+" "+second[i]
    dict_names[dfco1.columns[i]]= new
dfco1 = dfco1.rename(columns = dict_names).drop([0])

# show the first five records
dfco1.head()

,Survey Date,FIPS State,FIPS.1 County,Region Code,Division Code,County Name,County Bldgs,1-unit Units,1-unit Value,2-units Bldgs,2-units Units,2-units Value,3-4 units Bldgs,3-4 units Units,3-4 units Value,5+ units Bldgs,5+ units Units,5+ units Value,1-unit rep Bldgs,1-unit rep Units,1-unit rep Value,2-units rep Bldgs,2-units rep Units,2-units rep Value,3-4 units rep Bldgs,3-4 units rep Units,3-4 units rep Value,5+units rep Bldgs,5+units rep Units,5+units rep Value
1,2015,01,001,3,6,Autauga County,158,158,39749354,0,0,0,0,0,0,0,0,0,158,158,39749354,0,0,0,0,0,0,0,0,0
2,2015,01,003,3,6,Baldwin County,1622,1622,302576607,11,22,2232258,29,109,12724884,31,450,43856188,1336,1336,238194418,11,22,2232258,29,109,12724884,31,450,43856188
3,2015,01,005,3,6,Barbour County,10,10,3292300,0,0,0,0,0,0,0,0,0,10,10,3292300,0,0,0,0,0,0,0,0,0
4,2015,01,007,3,6,Bibb County,9,9,2222180,0,0,0,0,0,0,0,0,0,8,8,2108180,0,0,0,0,0,0,0,0,0
5,2015,01,009,3,6,Blount County,9,9,1573173,0,0,0,0,0,0,2,40,3831302,9,9,1573173,0,0,0,0,0,0,2,40,3831302


In [108]:
dfco2 = pd.read_table("https://www2.census.gov/econ/bps/County/co2016a.txt", sep=",",
                  skipinitialspace=True) #read the online table
                  # the web data table has some format issue, this block of code is to address that issue
dfco2["Unnamed 41"] = np.nan
dfco2 = dfco2.shift(1, axis=1)
dfco2.Survey = dfco2.index
dfco2.index = np.arange(len(dfco2))

dict_names = {}
second  = dfco2.iloc[0].fillna("")
for i in range(len(dfco2.columns)):
    first = dfco2.columns[i]
    if first.startswith("Unnamed"):
        if dfco2.columns[i-1].startswith("Unnamed"):
            first = dfco2.columns[i+1]
        else:
            first = dfco2.columns[i-1]
    #         new = first+second[i]
    # else:
    new = first+" "+second[i]
    dict_names[dfco2.columns[i]]= new
dfco2 = dfco2.rename(columns = dict_names).drop([0])

# show the first five records
dfco2.head()


,Survey Date,FIPS State,FIPS.1 County,Region Code,Division Code,County Name,County Bldgs,1-unit Units,1-unit Value,2-units Bldgs,2-units Units,2-units Value,3-4 units Bldgs,3-4 units Units,3-4 units Value,5+ units Bldgs,5+ units Units,5+ units Value,1-unit rep Bldgs,1-unit rep Units,1-unit rep Value,2-units rep Bldgs,2-units rep Units,2-units rep Value,3-4 units rep Bldgs,3-4 units rep Units,3-4 units rep Value,5+units rep Bldgs,5+units rep Units,5+units rep Value
1,2016,01,001,3,6,Autauga County,169,169,45256130,0,0,0,0,0,0,0,0,0,169,169,45256130,0,0,0,0,0,0,0,0,0
2,2016,01,003,3,6,Baldwin County,2137,2137,399000143,17,34,4525229,37,137,15089250,17,212,71612677,1996,1996,364360481,17,34,4525229,37,137,15089250,17,212,71612677
3,2016,01,005,3,6,Barbour County,3,3,686200,0,0,0,0,0,0,0,0,0,3,3,686200,0,0,0,0,0,0,0,0,0
4,2016,01,007,3,6,Bibb County,10,10,2355940,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,2016,01,009,3,6,Blount County,14,14,2815172,0,0,0,0,0,0,0,0,0,14,14,2815172,0,0,0,0,0,0,0,0,0


In [109]:
dfco2

,Survey Date,FIPS State,FIPS.1 County,Region Code,Division Code,County Name,County Bldgs,1-unit Units,1-unit Value,2-units Bldgs,2-units Units,2-units Value,3-4 units Bldgs,3-4 units Units,3-4 units Value,5+ units Bldgs,5+ units Units,5+ units Value,1-unit rep Bldgs,1-unit rep Units,1-unit rep Value,2-units rep Bldgs,2-units rep Units,2-units rep Value,3-4 units rep Bldgs,3-4 units rep Units,3-4 units rep Value,5+units rep Bldgs,5+units rep Units,5+units rep Value
1,2016,01,001,3,6,Autauga County,169,169,45256130,0,0,0,0,0,0,0,0,0,169,169,45256130,0,0,0,0,0,0,0,0,0
2,2016,01,003,3,6,Baldwin County,2137,2137,399000143,17,34,4525229,37,137,15089250,17,212,71612677,1996,1996,364360481,17,34,4525229,37,137,15089250,17,212,71612677
3,2016,01,005,3,6,Barbour County,3,3,686200,0,0,0,0,0,0,0,0,0,3,3,686200,0,0,0,0,0,0,0,0,0
4,2016,01,007,3,6,Bibb County,10,10,2355940,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,2016,01,009,3,6,Blount County,14,14,2815172,0,0,0,0,0,0,0,0,0,14,14,2815172,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3035,2016,56,037,4,8,Sweetwater County,70,70,19513641,1,2,290000,0,0,0,0,0,0,70,70,19513641,1,2,290000,0,0,0,0,0,0
3036,2016,56,039,4,8,Teton County,138,138,223351427,4,8,5395000,0,0,0,2,17,3296081,138,138,223351427,4,8,5395000,0,0,0,2,17,3296081
3037,2016,56,041,4,8,Uinta County,34,34,8079400,0,0,0,0,0,0,1,12,960000,34,34,8079400,0,0,0,0,0,0,1,12,960000
3038,2016,56,043,4,8,Washakie County,2,2,200000,1,2,200000,0,0,0,0,0,0,2,2,200000,1,2,200000,0,0,0,0,0,0


In [110]:
dfco3 = pd.read_table("https://www2.census.gov/econ/bps/County/co2017a.txt", sep=",",
                  skipinitialspace=True) #read the online table
                  # the web data table has some format issue, this block of code is to address that issue
dfco3["Unnamed 41"] = np.nan
dfco3 = dfco3.shift(1, axis=1)
dfco3.Survey = dfco3.index
dfco3.index = np.arange(len(dfco3))

dict_names = {}
second  = dfco3.iloc[0].fillna("")
for i in range(len(dfco3.columns)):
    first = dfco3.columns[i]
    if first.startswith("Unnamed"):
        if dfco3.columns[i-1].startswith("Unnamed"):
            first = dfco3.columns[i+1]
        else:
            first = dfco3.columns[i-1]
    #         new = first+second[i]
    # else:
    new = first+" "+second[i]
    dict_names[dfco3.columns[i]]= new
dfco3 = dfco3.rename(columns = dict_names).drop([0])

# show the first five records
dfco3.head()

,Survey Date,FIPS State,FIPS.1 County,Region Code,Division Code,County Name,County Bldgs,1-unit Units,1-unit Value,2-units Bldgs,2-units Units,2-units Value,3-4 units Bldgs,3-4 units Units,3-4 units Value,5+ units Bldgs,5+ units Units,5+ units Value,1-unit rep Bldgs,1-unit rep Units,1-unit rep Value,2-units rep Bldgs,2-units rep Units,2-units rep Value,3-4 units rep Bldgs,3-4 units rep Units,3-4 units rep Value,5+units rep Bldgs,5+units rep Units,5+units rep Value
1,2017,01,001,3,6,Autauga County,188,188,51711467,0,0,0,0,0,0,0,0,0,188,188,51711467,0,0,0,0,0,0,0,0,0
2,2017,01,003,3,6,Baldwin County,2207,2207,449821615,46,92,4240286,32,98,4034508,3,20,2091690,2160,2160,443998177,46,92,4240286,32,98,4034508,3,20,2091690
3,2017,01,005,3,6,Barbour County,3,3,901000,0,0,0,0,0,0,0,0,0,3,3,901000,0,0,0,0,0,0,0,0,0
4,2017,01,007,3,6,Bibb County,10,10,2551290,0,0,0,0,0,0,0,0,0,2,2,490000,0,0,0,0,0,0,0,0,0
5,2017,01,009,3,6,Blount County,18,18,4265412,0,0,0,0,0,0,0,0,0,18,18,4265412,0,0,0,0,0,0,0,0,0


In [111]:
import pandas as pd
import numpy as np
dfco4 = pd.read_table("https://www2.census.gov/econ/bps/County/co2018a.txt", sep=",",
                  skipinitialspace=True) #read the online table
                  # the web data table has some format issue, this block of code is to address that issue
dfco4["Unnamed 41"] = np.nan
dfco4 = dfco4.shift(1, axis=1)
dfco4.Survey = dfco4.index
dfco4.index = np.arange(len(dfco4))

dict_names = {}
second  = dfco4.iloc[0].fillna("")
for i in range(len(dfco4.columns)):
    first = dfco4.columns[i]
    if first.startswith("Unnamed"):
        if dfco4.columns[i-1].startswith("Unnamed"):
            first = dfco4.columns[i+1]
        else:
            first = dfco4.columns[i-1]
    #         new = first+second[i]
    # else:
    new = first+" "+second[i]
    dict_names[dfco4.columns[i]]= new
dfco4 = dfco4.rename(columns = dict_names).drop([0])

# show the first five records
dfco4.head()

,Survey Date,FIPS State,FIPS.1 County,Region Code,Division Code,County Name,County Bldgs,1-unit Units,1-unit Value,2-units Bldgs,2-units Units,2-units Value,3-4 units Bldgs,3-4 units Units,3-4 units Value,5+ units Bldgs,5+ units Units,5+ units Value,1-unit rep Bldgs,1-unit rep Units,1-unit rep Value,2-units rep Bldgs,2-units rep Units,2-units rep Value,3-4 units rep Bldgs,3-4 units rep Units,3-4 units rep Value,5+units rep Bldgs,5+units rep Units,5+units rep Value
1,2018,01,001,3,6,Autauga County,185,185,56148262,0,0,0,0,0,0,0,0,0,185,185,56148262,0,0,0,0,0,0,0,0,0
2,2018,01,003,3,6,Baldwin County,2459,2459,502077608,3,6,393370,3,12,1870926,28,570,110903040,2303,2303,463282763,3,6,393370,3,12,1870926,28,570,110903040
3,2018,01,005,3,6,Barbour County,6,6,1536900,0,0,0,0,0,0,0,0,0,6,6,1536900,0,0,0,0,0,0,0,0,0
4,2018,01,007,3,6,Bibb County,13,13,2422591,0,0,0,0,0,0,0,0,0,4,4,165500,0,0,0,0,0,0,0,0,0
5,2018,01,009,3,6,Blount County,7,7,1280871,0,0,0,1,4,223080,0,0,0,7,7,1280871,0,0,0,1,4,223080,0,0,0


In [112]:
dfco5 = pd.read_table("https://www2.census.gov/econ/bps/County/co2019a.txt", sep=",",
                  skipinitialspace=True) #read the online table
                  # the web data table has some format issue, this block of code is to address that issue
dfco5["Unnamed 41"] = np.nan
dfco5 = dfco5.shift(1, axis=1)
dfco5.Survey = dfco5.index
dfco5.index = np.arange(len(dfco5))

dict_names = {}
second  = dfco5.iloc[0].fillna("")
for i in range(len(dfco5.columns)):
    first = dfco5.columns[i]
    if first.startswith("Unnamed"):
        if dfco5.columns[i-1].startswith("Unnamed"):
            first = dfco5.columns[i+1]
        else:
            first = dfco5.columns[i-1]
    #         new = first+second[i]
    # else:
    new = first+" "+second[i]
    dict_names[dfco5.columns[i]]= new
dfco5 = dfco5.rename(columns = dict_names).drop([0])

# show the first five records
dfco5.head()

,Survey Date,FIPS State,FIPS.1 County,Region Code,Division Code,County Name,County Bldgs,1-unit Units,1-unit Value,2-units Bldgs,2-units Units,2-units Value,3-4 units Bldgs,3-4 units Units,3-4 units Value,5+ units Bldgs,5+ units Units,5+ units Value,1-unit rep Bldgs,1-unit rep Units,1-unit rep Value,2-units rep Bldgs,2-units rep Units,2-units rep Value,3-4 units rep Bldgs,3-4 units rep Units,3-4 units rep Value,5+units rep Bldgs,5+units rep Units,5+units rep Value
1,2019,01,001,3,6,Autauga County,242,242,69003879,0,0,0,0,0,0,0,0,0,242,242,69003879,0,0,0,0,0,0,0,0,0
2,2019,01,003,3,6,Baldwin County,2740,2740,571399246,21,42,5607750,6,22,2574702,10,346,42386042,2740,2740,571399246,21,42,5607750,6,22,2574702,10,346,42386042
3,2019,01,005,3,6,Barbour County,9,9,2261600,0,0,0,0,0,0,0,0,0,9,9,2261600,0,0,0,0,0,0,0,0,0
4,2019,01,007,3,6,Bibb County,14,14,2925717,0,0,0,1,3,280000,0,0,0,14,14,2925717,0,0,0,1,3,280000,0,0,0
5,2019,01,009,3,6,Blount County,25,25,4341017,0,0,0,1,4,175000,0,0,0,25,25,4341017,0,0,0,1,4,175000,0,0,0


# **Concatenating all years data**

In [113]:
vertical_concatcounty = pd.concat([dfco1,dfco2, dfco3,dfco4,dfco5], axis=0)

In [114]:
# Drop a specific column, e.g., 'Place Name'
vertical_concatcounty =vertical_concatcounty.reset_index()

In [115]:
vertical_concatcounty=vertical_concatcounty.drop(columns=['index'])

In [116]:
vertical_concatcounty

,Survey Date,FIPS State,FIPS.1 County,Region Code,Division Code,County Name,County Bldgs,1-unit Units,1-unit Value,2-units Bldgs,2-units Units,2-units Value,3-4 units Bldgs,3-4 units Units,3-4 units Value,5+ units Bldgs,5+ units Units,5+ units Value,1-unit rep Bldgs,1-unit rep Units,1-unit rep Value,2-units rep Bldgs,2-units rep Units,2-units rep Value,3-4 units rep Bldgs,3-4 units rep Units,3-4 units rep Value,5+units rep Bldgs,5+units rep Units,5+units rep Value
0,2015,01,001,3,6,Autauga County,158,158,39749354,0,0,0,0,0,0,0,0,0,158,158,39749354,0,0,0,0,0,0,0,0,0
1,2015,01,003,3,6,Baldwin County,1622,1622,302576607,11,22,2232258,29,109,12724884,31,450,43856188,1336,1336,238194418,11,22,2232258,29,109,12724884,31,450,43856188
2,2015,01,005,3,6,Barbour County,10,10,3292300,0,0,0,0,0,0,0,0,0,10,10,3292300,0,0,0,0,0,0,0,0,0
3,2015,01,007,3,6,Bibb County,9,9,2222180,0,0,0,0,0,0,0,0,0,8,8,2108180,0,0,0,0,0,0,0,0,0
4,2015,01,009,3,6,Blount County,9,9,1573173,0,0,0,0,0,0,2,40,3831302,9,9,1573173,0,0,0,0,0,0,2,40,3831302
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15189,2019,56,037,4,8,Sweetwater County,68,68,19672418,0,0,0,0,0,0,0,0,0,68,68,19672418,0,0,0,0,0,0,0,0,0
15190,2019,56,039,4,8,Teton County,122,122,185477431,7,14,4363233,3,9,349497,2,10,2080796,99,99,173455563,0,0,0,0,0,0,0,0,0
15191,2019,56,041,4,8,Uinta County,42,42,8549000,0,0,0,0,0,0,0,0,0,34,34,6789000,0,0,0,0,0,0,0,0,0
15192,2019,56,043,4,8,Washakie County,2,2,415000,0,0,0,0,0,0,0,0,0,2,2,415000,0,0,0,0,0,0,0,0,0


In [117]:
import pandas as pd


datacounty = vertical_concatcounty

# Select the columns of interest
selected_columnscounty = ['FIPS.1 County','Survey Date', '1-unit rep Units', 'County Name']

# Extract the desired data
extracted_datacounty = datacounty[selected_columnscounty]

In [118]:
extracted_datacounty

,FIPS.1 County,Survey Date,1-unit rep Units,County Name
0,001,2015,158,Autauga County
1,003,2015,1336,Baldwin County
2,005,2015,10,Barbour County
3,007,2015,8,Bibb County
4,009,2015,9,Blount County
...,...,...,...,...
15189,037,2019,68,Sweetwater County
15190,039,2019,99,Teton County
15191,041,2019,34,Uinta County
15192,043,2019,2,Washakie County


In [119]:
# Group by 'FIPS.1 County', 'Survey Date', 'County Name', and sum '1-unit rep Units'
grouped_df1county = extracted_datacounty.groupby(['FIPS.1 County', 'Survey Date', 'County Name'])['1-unit rep Units'].sum().reset_index()

# Pivot the DataFrame to have years as columns
pivot_df1county = grouped_df1county.pivot_table(index=['FIPS.1 County', 'County Name'], columns='Survey Date', values='1-unit rep Units', fill_value=0)

# Reset the index to have 'FIPS.1 County' and 'County Name' as columns
pivot_df1county.reset_index(inplace=True)

# Rename the columns
pivot_df1county.columns = ['FIPS.1 County', 'County Name', '2015', '2016', '2017', '2018', '2019']

# Display the pivot DataFrame
print(pivot_df1county)


     FIPS.1 County                     County Name  2015   2016   2017   2018  \
0              000  Maine Unorganized Territory        0    0.0    0.0    0.0   
1              000  Michigan Balance of State         51    0.0    0.0    0.0   
2              000  Montana Balance of State           0    0.0    0.0    0.0   
3              000  New Mexico Balance of State      168    0.0    0.0    0.0   
4              000  Oregon Balance of State           49    0.0  162.0    7.0   
...            ...                             ...   ...    ...    ...    ...   
2918           800  Suffolk (Independent City)       362  395.0  514.0  501.0   
2919           810  Virginia Beach (Independent Ci   696  768.0  646.0  534.0   
2920           820  Waynesboro (Independent City)     43   22.0   31.0   30.0   
2921           830  Williamsburg (Independent City    20   49.0   43.0   25.0   
2922           840  Winchester (Independent City)     13   25.0   18.0    0.0   

       2019  
0       0.0  

In [120]:
# Group by 'FIPS.1 County', 'Survey Date', 'County Name', and sum '1-unit rep Units'
grouped_df1county = extracted_datacounty[['FIPS.1 County', 'Survey Date', 'County Name', '1-unit rep Units']].groupby(['FIPS.1 County', 'Survey Date', 'County Name'])['1-unit rep Units'].sum().reset_index()

# Pivot the DataFrame to have years as columns
pivot_df1county = grouped_df1county.pivot_table(index=['FIPS.1 County', 'County Name'], columns='Survey Date', values='1-unit rep Units', fill_value=0)

# Reset the index to have 'FIPS.1 County' and 'County Name' as columns
pivot_df1county.reset_index(inplace=True)

# Rename the columns
pivot_df1county.columns = ['FIPS.1 County', 'County Name', '2015', '2016', '2017', '2018', '2019']



# Display the pivot DataFrame
print(pivot_df1county)


     FIPS.1 County                     County Name  2015   2016   2017   2018  \
0              000  Maine Unorganized Territory        0    0.0    0.0    0.0   
1              000  Michigan Balance of State         51    0.0    0.0    0.0   
2              000  Montana Balance of State           0    0.0    0.0    0.0   
3              000  New Mexico Balance of State      168    0.0    0.0    0.0   
4              000  Oregon Balance of State           49    0.0  162.0    7.0   
...            ...                             ...   ...    ...    ...    ...   
2918           800  Suffolk (Independent City)       362  395.0  514.0  501.0   
2919           810  Virginia Beach (Independent Ci   696  768.0  646.0  534.0   
2920           820  Waynesboro (Independent City)     43   22.0   31.0   30.0   
2921           830  Williamsburg (Independent City    20   49.0   43.0   25.0   
2922           840  Winchester (Independent City)     13   25.0   18.0    0.0   

       2019  
0       0.0  

# **County single units**

# **The Building permits for fips county code for all counties for single  units from 2015-2019 and csv file for it**


In [121]:
pivot_df1county

,FIPS.1 County,County Name,2015,2016,2017,2018,2019
0,000,Maine Unorganized Territory,0,0.0,0.0,0.0,0.0
1,000,Michigan Balance of State,51,0.0,0.0,0.0,0.0
2,000,Montana Balance of State,0,0.0,0.0,0.0,0.0
3,000,New Mexico Balance of State,168,0.0,0.0,0.0,0.0
4,000,Oregon Balance of State,49,0.0,162.0,7.0,402.0
...,...,...,...,...,...,...,...
2918,800,Suffolk (Independent City),362,395.0,514.0,501.0,500.0
2919,810,Virginia Beach (Independent Ci,696,768.0,646.0,534.0,667.0
2920,820,Waynesboro (Independent City),43,22.0,31.0,30.0,100.0
2921,830,Williamsburg (Independent City,20,49.0,43.0,25.0,29.0


In [122]:

# Save the pivot DataFrame to a CSV file
pivot_df1county.to_csv('3building_permitssingleunitcounty_grouped_2015-2019.csv', index=False)

In [123]:
import pandas as pd

datacounty2 = vertical_concatcounty

# Select the columns of interest
selected_columnscounty2 = ['FIPS.1 County', 'Survey Date', 'County Name',  '2-units rep Units','3-4 units rep Units','5+units rep Units']

# Extract the desired data
extracted_datamcounty2 = datacounty2[selected_columnscounty2]

In [124]:
extracted_datamcounty2

,FIPS.1 County,Survey Date,County Name,2-units rep Units,3-4 units rep Units,5+units rep Units
0,001,2015,Autauga County,0,0,0
1,003,2015,Baldwin County,22,109,450
2,005,2015,Barbour County,0,0,0
3,007,2015,Bibb County,0,0,0
4,009,2015,Blount County,0,0,40
...,...,...,...,...,...,...
15189,037,2019,Sweetwater County,0,0,0
15190,039,2019,Teton County,0,0,0
15191,041,2019,Uinta County,0,0,0
15192,043,2019,Washakie County,0,0,0


In [125]:
# Group by 'Zip Code,' 'FIPS Place Code,' 'Place Name,' and 'Survey Date,' and sum the units columns
grouped_dfmcounty = extracted_datamcounty2.groupby(['FIPS.1 County', 'Survey Date', 'County Name', ]).agg({
    '2-units rep Units': 'sum',
    '3-4 units rep Units': 'sum',
    '5+units rep Units': 'sum'
}).reset_index()

# Create a new column 'multi unit' by summing the '2-units rep Units,' '3-4 units rep Units,' and '5+ units rep Units'
grouped_dfmcounty['multi unit'] = grouped_dfmcounty['2-units rep Units'] + grouped_dfmcounty['3-4 units rep Units'] + grouped_dfmcounty['5+units rep Units']


# Display the grouped DataFrame
print(grouped_dfmcounty)

      FIPS.1 County Survey Date                     County Name  \
0               000        2015  Maine Unorganized Territory      
1               000        2015  Michigan Balance of State        
2               000        2015  Montana Balance of State         
3               000        2015  New Mexico Balance of State      
4               000        2015  Oregon Balance of State          
...             ...         ...                             ...   
14574           840        2015  Winchester (Independent City)    
14575           840        2016  Winchester (Independent City)    
14576           840        2017  Winchester (Independent City)    
14577           840        2018  Winchester (Independent City)    
14578           840        2019  Winchester (Independent City)    

      2-units rep Units 3-4 units rep Units 5+units rep Units multi unit  
0                     0                   0                 0        000  
1                     0                   0  

In [128]:
# Pivot the DataFrame to have years as columns
pivot_dfmcounty = grouped_dfmcounty.pivot_table(index=['FIPS.1 County', 'County Name'], columns='Survey Date', values='multi unit', fill_value=0)

# Reset the index to have 'Zip Code', 'FIPS Place Code', and 'Place Name' as columns
pivot_dfmcounty.reset_index(inplace=True)

# Rename the columns
pivot_dfmcounty.columns = ['FIPS.1 County', 'County Name', '2015', '2016', '2017', '2018', '2019']

# Save the pivot DataFrame to a CSV file
pivot_dfmcounty.to_csv('4building_permitscountymulti_grouped_2015-2019.csv', index=False)

# Display the pivot DataFrame
print(pivot_dfmcounty)

     FIPS.1 County                     County Name     2015   2016    2017  \
0              000  Maine Unorganized Territory         0.0    0.0     0.0   
1              000  Michigan Balance of State           0.0    0.0     0.0   
2              000  Montana Balance of State            0.0    0.0     0.0   
3              000  New Mexico Balance of State         0.0    0.0     0.0   
4              000  Oregon Balance of State             0.0    0.0     0.0   
...            ...                             ...      ...    ...     ...   
2918           800  Suffolk (Independent City)        436.0  312.0    48.0   
2919           810  Virginia Beach (Independent Ci  12785.0  815.0   877.0   
2920           820  Waynesboro (Independent City)       0.0    0.0     0.0   
2921           830  Williamsburg (Independent City      0.0    0.0     0.0   
2922           840  Winchester (Independent City)       0.0    5.0  8144.0   

       2018     2019  
0       0.0      0.0  
1       0.0      

# **The Building permits for fips county code for all counties for multiple units from 2015-2019 and csv file for it**

In [127]:
pivot_dfmcounty

,FIPS.1 County,County Name,2015,2016,2017,2018,2019
0,000,Maine Unorganized Territory,0.0,0.0,0.0,0.0,0.0
1,000,Michigan Balance of State,0.0,0.0,0.0,0.0,0.0
2,000,Montana Balance of State,0.0,0.0,0.0,0.0,0.0
3,000,New Mexico Balance of State,0.0,0.0,0.0,0.0,0.0
4,000,Oregon Balance of State,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...
2918,800,Suffolk (Independent City),436.0,312.0,48.0,216.0,195.0
2919,810,Virginia Beach (Independent Ci,12785.0,815.0,877.0,245.0,683.0
2920,820,Waynesboro (Independent City),0.0,0.0,0.0,276.0,20120.0
2921,830,Williamsburg (Independent City,0.0,0.0,0.0,0.0,0.0
